# 数据预处理

In [1]:
import numpy as np
import pickle
from collections import defaultdict
import sys, re
import pandas as pd
import os
import csv
import glob
import random
# import gensim
from sklearn.model_selection import train_test_split
import string

import inflection
import nltk
from nltk.stem.porter import PorterStemmer

from assets import stop_words, java_keywords
from parsers import Parser
from collections import OrderedDict
# from datasets import DATASET
from collections import namedtuple
from pathlib import Path

In [3]:
bug_ids=[]
summarys=[]
descriptions=[]
report_times=[]
commits=[]
files_list=[]
choosed_files=[]
with open('/data/hdj/SourceFile/data/sourceFile_aspectj/bugreport.txt','r',encoding='utf-8') as f_in:
    for i, line in enumerate(f_in) :
        if i==0:
            continue
#         print(line.strip().split('\t'))
        contents=line.strip().split('\t')
        bug_ids.append(contents[1])
        summarys.append(contents[2])
        descriptions.append(contents[3])
        report_times.append(contents[4])
        commits.append(contents[7])
        #处理files，添加commit
        fixed_list=[]
        for file_path in contents[9].split(' '):
            if ':' in file_path:
                continue
            str_list=file_path.split('/')
            str_list[-1]=contents[7]+' '+str_list[-1]
#             print(str_list)
            new_string='/'.join(str_list)
            fixed_list.append(new_string)
        files_list.append(fixed_list)
        choosed_files.extend(fixed_list)#用作生成neg的采样库
#         if i==3:
#             break

In [52]:
choosed_files[:3]

['org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/core/builder/dd88d21 AjState.java',
 'tests/bugs175/pr423257/dd88d21 AspectX.java',
 'tests/bugs175/pr423257/dd88d21 Test.java']

In [48]:
aspectj_src_addresses = glob.glob(str('/data/hdj/SourceFile/data/sourceFile_aspectj/org.aspectj') + '/**/*.java', recursive=True)
len(aspectj_src_addresses)

2394

In [54]:
aspectj_clean_src_addresses = glob.glob(str('/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj') + '/**/*.java', recursive=True)
len(aspectj_src_addresses)


2357

In [55]:
choosed_files_in_clean=[]
for file in choosed_files:
    if '/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/'+file  in aspectj_clean_src_addresses:
        choosed_files_in_clean.append(file)
# aspectj_src_addresses[:2]

In [57]:
len(choosed_files_in_clean)
choosed_files=choosed_files_in_clean

In [58]:
len(choosed_files),choosed_files[:10]

(2357,
 ['org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/core/builder/dd88d21 AjState.java',
  'tests/bugs175/pr423257/dd88d21 AspectX.java',
  'tests/bugs175/pr423257/dd88d21 Test.java',
  'tests/src/org/aspectj/systemtest/dd88d21 AllTests17.java',
  'tests/src/org/aspectj/systemtest/ajc175/dd88d21 Ajc175Tests.java',
  'tests/src/org/aspectj/systemtest/ajc175/dd88d21 AllTestsAspectJ175.java',
  'tests/src/org/aspectj/systemtest/incremental/tools/dd88d21 MultiProjectIncrementalTests.java',
  'weaver/src/org/aspectj/weaver/bcel/asm/dd88d21 StackMapAdder.java',
  'org.aspectj.matcher/src/org/aspectj/weaver/9319e34 Checker.java',
  'tests/bugs174/extra_inserts/9319e34 Code.java'])

In [18]:
#改成dataframe格式
all_data=pd.DataFrame()
all_data['bug_id']=bug_ids
all_data['summary']=summarys
all_data['description']=descriptions
all_data['report_time']=report_times
all_data['commit']=commits
all_data['files']=files_list
#按照时间升序排列
def remove_twoHeadWord(string):
    contents=string.split(' ')[2:]
    return ' '.join(contents)
all_data['summary_remove2']=all_data['summary'].apply(remove_twoHeadWord)
all_data.sort_values("report_time",inplace=True)
all_data.to_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/all_data.csv',index=False)

In [21]:
reportDict=OrderedDict()
ids=list(all_data['bug_id'].values)
summary_remove2=list(all_data['summary_remove2'].values)
reportLen=[]
for key,value in zip(ids,summary_remove2):
    reportDict[key]=value
    reportLen.append(len(value.split(' ')))
reportLen=sorted(reportLen)
print('总共文件数 :', len(reportLen), reportLen)
a = np.array(reportLen)
# 25%分位数
print('report 的 10%',np.percentile(a, 10))
# 75%分位数
print('20 :',np.percentile(a, 20))
print('40 :',np.percentile(a, 40))
# 中位数
print('中位数 :',np.median(a))
print('60 :',np.percentile(a, 60))
print('80 :',np.percentile(a, 80))
print('90 :',np.percentile(a, 90))
#拼接成 /data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/zxing_report_pos_summ_descr.pickle
with open('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_report.pickle', 'wb') as file:
    pickle.dump(reportDict, file, protocol=pickle.HIGHEST_PROTOCOL)

总共文件数 : 593 [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 

In [23]:
# reportDict

In [19]:
all_data.reset_index(inplace=True,drop=True)
all_data.head()

,bug_id,summary,description,report_time,commit,files,summary_remove2
0,11280,Bug 11280 [View Mgmt] Switching perspectives l...,1. Open the CVS repository view in the Java pe...,2002-03-13 12:40:54,508dbcb,[tests/src/org/aspectj/systemtest/ajc150/508db...,[View Mgmt] Switching perspectives loses view ...
1,28919,Bug 28919 waever tries to weave into native me...,"""If you don't find the exception below in a bu...",2002-12-30 16:40:03,2781720,[weaver/src/org/aspectj/weaver/bcel/2781720 La...,waever tries to weave into native methods ...
2,28974,"""Bug 28974 Compiler error when introducing a ""...","""The aspect below fails to compile with 1.1b2,...",2003-01-03 10:28:23,99a873c,[weaver/src/org/aspectj/weaver/99a873c AjcMemb...,"Compiler error when introducing a """"final"""" fi..."
3,29186,Bug 29186 ajc -emacssym chokes on pointcut tha...,,2003-01-08 21:22:48,c1260e6,[asm/src/org/aspectj/asm/c1260e6 StructureMode...,ajc -emacssym chokes on pointcut that includes...
4,29662,Bug 29662 VerifyError on accessing objects not...,"""I use aspectj1.1b4 (the version list box did ...",2003-01-16 15:12:47,2d297d7,[tests/bugs/2d297d7 AroundAccess.java],VerifyError on accessing objects not accessibl...


In [12]:
' '.join(all_data['summary'][0].split(' ')[2:])

'[View Mgmt] Switching perspectives loses view maximized state'

In [17]:
all_data['summary_remove2'][:4]

0    [View Mgmt] Switching perspectives loses view ...
1        waever tries to weave into native methods ...
2    Compiler error when introducing a ""final"" fi...
3    ajc -emacssym chokes on pointcut that includes...
Name: summary_remove2, dtype: object

In [36]:
#分割成70%train  10%val 20%val
print(all_data.shape[0])
savePath='/data/hdj/SourceFile/data/sourceFile_aspectj/'
project='aspectj_'
train_len=int(all_data.shape[0]*0.7)
val_len=int(all_data.shape[0]*0.1)
train_val_len=int(all_data.shape[0]*0.7)+int(all_data.shape[0]*0.1)
print(train_len,val_len,train_val_len,all_data.shape[0]-train_val_len)
train=all_data[:train_len]
val=all_data[train_len:train_len+val_len]
test=all_data[train_len+val_len:]
print(train.shape,val.shape,test.shape)
train.to_csv(savePath+project+'train.csv',index=False)
val.to_csv(savePath+project+'val.csv',index=False)
test.to_csv(savePath+project+'test.csv',index=False)

593
415 59 474 119
(415, 7) (59, 7) (119, 7)


In [38]:
train.head()

,bug_id,summary,description,report_time,commit,files,summary_remove2
0,11280,Bug 11280 [View Mgmt] Switching perspectives l...,1. Open the CVS repository view in the Java pe...,2002-03-13 12:40:54,508dbcb,[tests/src/org/aspectj/systemtest/ajc150/508db...,[View Mgmt] Switching perspectives loses view ...
1,28919,Bug 28919 waever tries to weave into native me...,"""If you don't find the exception below in a bu...",2002-12-30 16:40:03,2781720,[weaver/src/org/aspectj/weaver/bcel/2781720 La...,waever tries to weave into native methods ...
2,28974,"""Bug 28974 Compiler error when introducing a ""...","""The aspect below fails to compile with 1.1b2,...",2003-01-03 10:28:23,99a873c,[weaver/src/org/aspectj/weaver/99a873c AjcMemb...,"Compiler error when introducing a """"final"""" fi..."
3,29186,Bug 29186 ajc -emacssym chokes on pointcut tha...,,2003-01-08 21:22:48,c1260e6,[asm/src/org/aspectj/asm/c1260e6 StructureMode...,ajc -emacssym chokes on pointcut that includes...
4,29662,Bug 29662 VerifyError on accessing objects not...,"""I use aspectj1.1b4 (the version list box did ...",2003-01-16 15:12:47,2d297d7,[tests/bugs/2d297d7 AroundAccess.java],VerifyError on accessing objects not accessibl...


In [34]:
#读取所有的文件
aspectj_src_addresses = glob.glob(str('/data/hdj/SourceFile/data/sourceFile_aspectj/org.aspectj') + '/**/*.java', recursive=True)
eclipse_src_addresses = glob.glob(str('/data/hdj/SourceFile/data/sourceFile_eclipseUI/eclipse.platform.ui') + '/**/*.java', recursive=True)
jdt_src_addresses = glob.glob(str('/data/hdj/SourceFile/data/sourceFile_jdt/eclipse.jdt.ui') + '/**/*.java', recursive=True)
swt_src_addresses = glob.glob(str('/data/hdj/SourceFile/data/sourceFile_swt/eclipse.platform.swt') + '/**/*.java', recursive=True)
tomcat_src_addresses = glob.glob(str('/data/hdj/SourceFile/data/sourceFile_tomcat/tomcat') + '/**/*.java', recursive=True)

print(len(aspectj_src_addresses),len(eclipse_src_addresses),len(jdt_src_addresses),len(swt_src_addresses),len(tomcat_src_addresses))

2394 17809 16302 8560 2567


In [70]:
train=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_train.csv')
val=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_val.csv')

train.head()

,bug_id,summary,description,report_time,commit,files
0,11280,Bug 11280 [View Mgmt] Switching perspectives l...,1. Open the CVS repository view in the Java pe...,2002-03-13 12:40:54,508dbcb,['tests/src/org/aspectj/systemtest/ajc150/508d...
1,28919,Bug 28919 waever tries to weave into native me...,"""If you don't find the exception below in a bu...",2002-12-30 16:40:03,2781720,['weaver/src/org/aspectj/weaver/bcel/2781720 L...
2,28974,"""Bug 28974 Compiler error when introducing a ""...","""The aspect below fails to compile with 1.1b2,...",2003-01-03 10:28:23,99a873c,['weaver/src/org/aspectj/weaver/99a873c AjcMem...
3,29186,Bug 29186 ajc -emacssym chokes on pointcut tha...,NaN,2003-01-08 21:22:48,c1260e6,['asm/src/org/aspectj/asm/c1260e6 StructureMod...
4,29662,Bug 29662 VerifyError on accessing objects not...,"""I use aspectj1.1b4 (the version list box did ...",2003-01-16 15:12:47,2d297d7,['tests/bugs/2d297d7 AroundAccess.java']


In [39]:
val.head()

,bug_id,summary,description,report_time,commit,files,summary_remove2
415,152835,Bug 152835 ArrayIndexOutOfBoundsException in E...,Got the following exception during startup of ...,2006-08-04 06:00:33,908b405,[org.aspectj.ajdt.core/src/org/aspectj/ajdt/in...,ArrayIndexOutOfBoundsException in EclipseAdapt...
416,152873,Bug 152873 Optimize shouldWeaveAnnotationStyle...,,2006-08-04 15:21:07,f239f2a,[testing/newsrc/org/aspectj/testing/f239f2a An...,Optimize shouldWeaveAnnotationStyleAspect with...
417,152979,Bug 152979 Optimization: Reference Use and URL...,"""I have found that a lot of memory can get pin...",2006-08-07 11:04:12,387c3ac,[bcel-builder/src/org/aspectj/apache/bcel/util...,Optimization: Reference Use and URL Caching in...
418,153535,Bug 153535 Bug in reflection delegate signatur...,"""The following problem is interesting because ...",2006-08-11 03:34:59,82e3e13,[weaver/src/org/aspectj/weaver/reflect/82e3e13...,Bug in reflection delegate signature for array...
419,153845,Bug 153845 [generics] Problem with signature f...,"""public aspect OuterAspect { private pointcut ...",2006-08-14 21:28:08,70ae0f8,"[tests/bugs153/pr153845/70ae0f8 Aspect.java, t...",[generics] Problem with signature for generic ...


In [44]:
for index, row in train.iterrows():
        print(row['files'])
        print(type(row['files']))
        print(index)
#         files=eval(row['files'])
        files_before=eval(row['files'])

['tests/src/org/aspectj/systemtest/ajc150/508dbcb Ajc150Tests.java', 'weaver/src/org/aspectj/weaver/reflect/508dbcb ReflectionBasedReferenceTypeDelegateFactory.java']
<class 'list'>
0


TypeError: eval() arg 1 must be a string, bytes or code object

In [59]:
def remove_not_exist_code(fixed_files,exists_files):
    '''
        fixed_files: list 修复文件
        exists_files: list 清洗后仍然存在的文件
    '''
    #去除文件为空

    fixed=[file for file in fixed_files if file in exists_files]
    return fixed
    #去除完成
def generate_train_val_pos_neg_logit_data(prepath,project_name,data,sample_num,dtype='train'):
    '''
        prepath：输出的路径 /data/hdj/SourceFile/data/sourceFile_aspectj
        project_name:项目名 aspectj_
        data:dataframe 数据
        sample_num:采样数量
        dtype: train val
        
    '''
    out_ids = []
    out_neg_ids=[]
    out_pos_path = []
    out_neg_path = []
    for index, row in data.iterrows():
    #     print(row)
#         print(index)
#         files=eval(row['files'])
        files_before=row['files']
        files=remove_not_exist_code(files_before,choosed_files)#去除掉不存在的fixed_files
    #     print(files)
        fix_name_list=[]
#         print('files :',files)
        for fix_one in files:
            #正样本构造
            out_ids.append(row['bug_id'])
            out_pos_path.append(fix_one)
            #正样本构造完成
#             print('fix_one :',fix_one)
            name=fix_one.split('/')[-1].split(' ')[1]
            fix_name_list.append(name)
        #过滤掉fixed_files里同名但不同版本的文件
        Neg_files=[]
        for to_choose in choosed_files:
            flag=True
            for fix_name in fix_name_list:
                if fix_name in to_choose:
                    flag=False
                    break
            if flag:
                Neg_files.append(to_choose)
        #过滤完成
        #开始采样
        neg_sample=sample_num-len(fix_name_list)
        if neg_sample>len(Neg_files):
            neg_sample=len(Neg_files)

        Neg_files_k = random.sample(Neg_files, k=neg_sample)
#         print('采样数量 ：',len(Neg_files_k),' 总共文件数量 :',len(Neg_files))
        #采样完成
        #负样本构造
        for neg_file in Neg_files_k:
            out_neg_ids.append(row['bug_id'])
            out_neg_path.append(neg_file)
        #负样本构造完成
#         if index==1:
#             break
    #     print('index :',index)
    train_pos = pd.DataFrame()
    train_neg = pd.DataFrame()
    train_pos['id'] = out_ids
    train_pos['path'] = out_pos_path
    train_pos['label'] = 1

    train_neg['id'] = out_neg_ids
    train_neg['path'] = out_neg_path
    train_neg['label'] = 0
    print(dtype+' :',len(out_ids), len(out_pos_path), len(out_neg_ids),len(out_neg_path))
    if dtype=='train':
        train_pos.to_csv(prepath + project_name + 'training_pos.csv', index=False)
        train_neg.to_csv(prepath + project_name + 'training_neg.csv', index=False)
    else:
        train_pos.to_csv(prepath + project_name + 'val_pos.csv', index=False)
        train_neg.to_csv(prepath + project_name + 'val_neg.csv', index=False)
        
#训练和验证的数据生成
generate_train_val_pos_neg_logit_data(savePath,project,train,sample_num=300,dtype='train')
generate_train_val_pos_neg_logit_data(savePath,project,val,sample_num=300,dtype='val')
#训练和验证数据生成结束

train : 1852 1852 122648 122648
val : 184 184 17516 17516


In [65]:

def generate_test_pos_neg_logit_data(prepath,projct_name,data,sample_num=1000,dtype='test'):
    '''
        prepath：输出的路径 /data/hdj/SourceFile/data/sourceFile_aspectj/
        project_name:项目名 aspectj_
        data:dataframe 数据
        sample_num:采样数量
        dtype: test
        
    '''
    #生成test开始
#     choosed_files=readAll(os.path.join(filter_path,project_names[i]+'filter.txt'))
    out_ids=[]
    out_path=[]
    out_label=[]
    for index, row in data.iterrows():
        files_before=eval(row['files'])
        files=remove_not_exist_code(files_before,choosed_files)#去除掉不存在的fixed_files
#         print('删除fixed_files前',len(files_before),'删除不存在的files还剩',len(files))
        fix_name_list=[]
        for fix_one in files:
            #正样本构造
            out_ids.append(row['bug_id'])
            out_path.append(fix_one)
            out_label.append(1)
            #正样本构造完成
#             print('fix_one :',fix_one)
            name=fix_one.split('/')[-1].split(' ')[1]
            fix_name_list.append(name)
        #过滤掉fixed_files里同名但不同版本的文件
        Neg_files=[]
        for to_choose in choosed_files:
            flag=True
            for fix_name in fix_name_list:
                if fix_name in to_choose:
                    flag=False
                    break
            if flag:
                Neg_files.append(to_choose)
        #过滤完成
        neg_sample=sample_num-len(files)
        if neg_sample>len(Neg_files):
            neg_sample=len(Neg_files)
        Neg_files_k = random.sample(Neg_files, k=neg_sample)
        if len(files)==0:
            print("test 的fixed files数量为0 :",id)
            continue
        for unfixed in Neg_files_k:
            out_ids.append(row['bug_id'])
            out_path.append(unfixed)
            out_label.append(0)

    out_data=pd.DataFrame()
    out_data['id']=out_ids
    out_data['path']=out_path
    out_data['label']=out_label
    out_data.to_csv(prepath+projct_name+'test_pos_neg.csv' ,index=False)
test=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_test.csv')
generate_test_pos_neg_logit_data(savePath,project,test,sample_num=1000,dtype='test')

In [102]:
aspectj_train_pos=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_training_pos.csv')
aspectj_train_neg=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_training_neg.csv')

aspectj_train_pos.head()

,id,path,label
0,11280,tests/src/org/aspectj/systemtest/ajc150/508dbc...,1
1,11280,weaver/src/org/aspectj/weaver/reflect/508dbcb ...,1
2,28919,weaver/src/org/aspectj/weaver/bcel/2781720 Laz...,1
3,28974,weaver/src/org/aspectj/weaver/99a873c AjcMembe...,1
4,29186,asm/src/org/aspectj/asm/c1260e6 StructureModel...,1


In [118]:
aspectj_test_pos_neg=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_test_pos_neg.csv')
aspectj_test_pos_neg.head()

,id,path,label
0,251326,tests/bugs163/pr251326/pkgA/f1a83b5 Listener.java,1
1,251326,tests/bugs163/pr251326/pkgA/f1a83b5 Target.java,1
2,251326,tests/src/org/aspectj/systemtest/ajc163/f1a83b...,1
3,251326,tests/bugs172/pr398588/96ebaae Blah.java,0
4,251326,weaver/src/org/aspectj/weaver/patterns/7d14432...,0


# 生成物理上的数据

In [28]:
def code_pre(outstring):
    #主要目的是去除所有的注释 ，import， 替换回车换行符 ，过滤掉词长度小于3，最终返回数据格式是list
    m = re.compile(r'/\*.*?\*/', re.S)
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m = re.compile(r'//.*')
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m=re.compile(r'import.*')
    outtmp=re.sub(m,'',outstring)
    outstring = outtmp
    m = re.compile(r'package.*')
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m = re.compile(r'#.*')
    outtmp = re.sub(m, ' ', outstring)
    outstring = outtmp
    for char in ['\r\n', '\r', '\n']:
        outstring = outstring.replace(char, ' ')
    list_str=outstring.split()
    list_str=[t for t in list_str if len(t)>=3]
    # outstring = ' '.join(list_str)
    return list_str

In [27]:
def get_example(spamreader,examples,report_data,clean_path,codeDict,reportDict):#codeDict查询文件对应的class_method_names
    '''
        spamreader,读取的是该文件里的数据data/hdj/cross_project_trans/report/aspectj_80_training_pos.csv 格式：
            id,28974
            path,org.aspectj/modules/weaver/src/org/aspectj/weaver/patterns/ITokenSource.java
            label，0
        report_data,读取的是该文件 /data/hdj/cross_project_trans/report/aspectj_report_clean.csv，得到clean后的report信息
        clean_path,clean后的code的目录地址'/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/'
        codeDict,#单独提取code的className 和methodName信息，是dict格式
        reportDict,#单独提取的report的带词性的summary 和description的信息，dict格式
        
        return examples,返回生成的样本
    '''
    i = 0
    for row in spamreader:
        if i == 0:#为了跳过首行的标题
            i += 1
            continue
        # bug_report_name = "JDT_CNN/BugCorpus/BugCorpus/" + row[0].strip()
        # 根据bugid获得对应report
        bug_report_id = row[0].strip()
        # bug_description = getReport(bug_report_id, report_data)
        pos_summ_descrip=reportDict[bug_report_id]
        source_code_name = row[1].strip()
        #拼装成 /home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/org.aspectj/modules/weaver/src/org/aspectj/weaver/patterns/ITokenSource.java
        code_path = clean_path + source_code_name
        try:
            rf = open(code_path, 'r', encoding='utf-8', errors='ignore')
            data = rf.read()
        except:
            # print(data)
#             rf.close()
            print(code_path,'不存在*******')
            continue
        
        try:#如果code可以使用ast解析出来就使用className和methodName
            code=codeDict[source_code_name]+' '
        except:#否则使用原始的code文本数据
            # code=''
            code = ' '.join(code_pre(data))
        # code += ' '.join(code_pre(data))

        label = row[2].strip()
        tmp = []
        tmp.append(label)
        tmp.append(bug_report_id)  # TODO 记录report_id
        tmp.append(source_code_name)
        tmp.append(pos_summ_descrip)
        tmp.append(code)
        #数据格式是0<CODESPLIT>95536<CODESPLIT>org.aspectj/modules/weaver/src/org/aspectj/weaver/patterns/ITokenSource.java<CODESPLIT>pos_summ_descrip<CODESPLIT>code
        examples.append("<CODESPLIT>".join(tmp))
    return examples


In [63]:
def load_data( report_path, project_name, more_project_name, clean_path,tag,codeDict,reportDict):
    ''' 
        report_path, '/data/hdj/SourceFile/data/sourceFile_aspectj/'
        project_name, aspectj_
        more_project_name, ''
        clean_path,'/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/'
        tag,'train'
        codeDict,dict格式，存放code的单独提取出来的信息
        reportDict，dict格式，单独提取的report的带词性的summary 和description的信息，
    '''
    # labels_list, report_ids_list, path_list,report_list,code_list = [], [], [],[],[]
    examples=[]
    #拼装成该目录 /data/hdj/cross_project_trans/report/aspectj_report_clean.csv，得到clean后的report信息
#     report_data = pd.read_csv(report_path + project_name + 'report_clean.csv')
    report_data=None
    #读取pos的数据
    if tag=='train':
    #拼装成/data/hdj/cross_project_trans/report/aspectj_80_training_pos.csv
        pos_spamreader = csv.reader(open(report_path + project_name + "training_pos.csv", newline=''))
        examples=get_example(pos_spamreader,examples,report_data,clean_path,codeDict,reportDict)
        #读取neg的数据
        #拼装成/data/hdj/cross_project_trans/report/aspectj_80_training_neg.csv
        neg_spamreader = csv.reader(open(report_path + project_name + "training_neg.csv", newline=''))
        examples = get_example(neg_spamreader, examples, report_data, clean_path,codeDict,reportDict)
    elif tag=='val':
        pos_spamreader = csv.reader(open(report_path + project_name + "training_pos.csv", newline=''))
        examples=get_example(pos_spamreader,examples,report_data,clean_path,codeDict,reportDict)
        #读取neg的数据
        #拼装成/data/hdj/cross_project_trans/report/aspectj_80_training_neg.csv
        neg_spamreader = csv.reader(open(report_path + project_name + "val_neg.csv", newline=''))
        examples = get_example(neg_spamreader, examples, report_data, clean_path,codeDict,reportDict)
    else:
        pos_spamreader = csv.reader(open(report_path + project_name + "test_pos_neg.csv", newline=''))
        examples=get_example(pos_spamreader,examples,report_data,clean_path,codeDict,reportDict)
    #aspectj_train.txt
    #保存数据
    if tag=='train':
#         train_set, val_set = train_test_split(examples, test_size=0.2, random_state=42)
        #拼装成 /data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_train.txt
        with open('/data/hdj/SourceFile/data/sourceFile_aspectj/train_valid/'+project_name+tag+'.txt','w',encoding='utf-8') as f_out:
            for example in examples:
                f_out.write(example+'\n')
        #/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_val.txt
    elif tag=='val':
        with open('/data/hdj/SourceFile/data/sourceFile_aspectj/train_valid/'+project_name+'val'+'.txt','w',encoding='utf-8') as f_out:
            for example in examples:
                f_out.write(example+'\n')
    elif tag=='test':
        with open('/data/hdj/SourceFile/data/sourceFile_aspectj/train_valid/'+project_name+'test'+'.txt','w',encoding='utf-8') as f_out:
            for example in examples:
                f_out.write(example+'\n')
#     else:
#         #/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_test.txt 这里未必在使用，应该单独写好了test集的生成过程
#         with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/'+project_name+tag+'.txt','w',encoding='utf-8') as f_out:
#             for example in examples:
#                 f_out.write(example+'\n')

In [60]:
load_data('/data/hdj/SourceFile/data/sourceFile_aspectj/','aspectj_',''
          ,'/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/','train'
         ,None,reportDict)


In [61]:
load_data('/data/hdj/SourceFile/data/sourceFile_aspectj/','aspectj_',''
          ,'/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/','val'
         ,None,reportDict)


In [66]:
load_data('/data/hdj/SourceFile/data/sourceFile_aspectj/','aspectj_',''
          ,'/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/','test'
         ,None,reportDict)


In [108]:
aspectj_train_pos.shape,aspectj_train_neg.shape

((1879, 3), (122621, 3))

In [88]:
train_pos.head()

,id,path,label
0,11280,tests/src/org/aspectj/systemtest/ajc150/508dbc...,1
1,11280,weaver/src/org/aspectj/weaver/reflect/508dbcb ...,1
2,28919,weaver/src/org/aspectj/weaver/bcel/2781720 Laz...,1


In [89]:
train_neg.head()

,id,path,label
0,11280,tests/src/org/aspectj/systemtest/ajc150/ltw/1e...,0
1,11280,tests/src/org/aspectj/systemtest/ajc153/bebb36...,0
2,11280,weaver/src/org/aspectj/weaver/952dda9 TypeX.java,0
3,11280,weaver/src/org/aspectj/weaver/patterns/be03167...,0
4,11280,testing/src/org/aspectj/testing/harness/bridge...,0


In [109]:
# len(out_neg_ids),out_neg_ids

In [ ]:
#分割成70%train  10%val 20%val
train_len=len()

In [11]:
print(all_data.shape)
all_data.tail(10)

(593, 6)

In [16]:
# bug_ids
# summarys
# descriptions
# report_times
# commits
# files_list

2394 17809 16302 8560 2567


In [5]:
for path_name in aspectj_src_addresses:
    if 'WeavingURLClassLoaderTest.java' in path_name:
        print(path_name)

/data/hdj/SourceFile/data/sourceFile_aspectj/org.aspectj/weaver/testsrc/org/aspectj/weaver/33d8ee9 WeavingURLClassLoaderTest.java
/data/hdj/SourceFile/data/sourceFile_aspectj/org.aspectj/weaver/testsrc/org/aspectj/weaver/9ebfc84 WeavingURLClassLoaderTest.java
/data/hdj/SourceFile/data/sourceFile_aspectj/org.aspectj/weaver/testsrc/org/aspectj/weaver/edd6539 WeavingURLClassLoaderTest.java
/data/hdj/SourceFile/data/sourceFile_aspectj/org.aspectj/weaver/testsrc/org/aspectj/weaver/0521e79 WeavingURLClassLoaderTest.java


In [5]:

for src_address in [swt_src_addresses,tomcat_src_addresses]:#aspectj_src_addresses,eclipse_src_addresses,jdt_src_addresses,
    fileName=set()
    print(len(src_address))
    for path_name in src_address:
        tmp=path_name.split('/')[-1].split(' ')[1]
        fileName.add(tmp)
    print(len(fileName))
    break

8560
922


In [7]:
for path_name in swt_src_addresses:
    if 'ACC.java' in path_name:
        print(path_name)

/data/hdj/SourceFile/data/sourceFile_swt/eclipse.platform.swt/tests/org.eclipse.swt.tests/JUnit Tests/org/eclipse/swt/tests/junit/6938a0d Test_org_eclipse_swt_accessibility_ACC.java
/data/hdj/SourceFile/data/sourceFile_swt/eclipse.platform.swt/bundles/org.eclipse.swt/Eclipse SWT Accessibility/common/org/eclipse/swt/accessibility/6694f7a ACC.java
/data/hdj/SourceFile/data/sourceFile_swt/eclipse.platform.swt/bundles/org.eclipse.swt/Eclipse SWT Accessibility/common/org/eclipse/swt/accessibility/54340ef ACC.java
/data/hdj/SourceFile/data/sourceFile_swt/eclipse.platform.swt/bundles/org.eclipse.swt/Eclipse SWT Accessibility/common/org/eclipse/swt/accessibility/6b3c7f0 ACC.java
/data/hdj/SourceFile/data/sourceFile_swt/eclipse.platform.swt/bundles/org.eclipse.swt/Eclipse SWT Accessibility/common/org/eclipse/swt/accessibility/a3ce480 ACC.java


In [6]:
fileName

{'ACC.java',
 'ACTCTX.java',
 'AGL.java',
 'ASTClass.java',
 'ATK.java',
 'Accessible.java',
 'AccessibleAttributeAdapter.java',
 'AccessibleAttributeEvent.java',
 'AccessibleAttributeListener.java',
 'AccessibleFactory.java',
 'AccessibleObject.java',
 'AccessibleTextEvent.java',
 'AddressBook.java',
 'AllBrowserTests.java',
 'AllCarbonTests.java',
 'AllTests.java',
 'AllWidgetTests.java',
 'AppFileLocProvider.java',
 'AtkActionIface.java',
 'AtkObjectClass.java',
 'AtkTableIface.java',
 'AuthenticationEvent.java',
 'AuthenticationListener.java',
 'BidiSegmentEvent.java',
 'BidiUtil.java',
 'Browser.java',
 'Browser1.java',
 'Browser2.java',
 'Browser3.java',
 'Browser4.java',
 'Browser5.java',
 'Browser6.java',
 'Browser7.java',
 'Browser8.java',
 'Browser9.java',
 'BrowserDemoView.java',
 'BrowserFactory.java',
 'BrowserFunction.java',
 'BrowserTab.java',
 'Bullet.java',
 'BusyIndicator.java',
 'Button.java',
 'ButtonDrawData.java',
 'ButtonTab.java',
 'ByteArrayTransfer.java',
 'CB

In [9]:
eclipse_src_addresses[0].split('/')[-1].split(' ')[1]

'IdClassLabelColorTest.java'

# 逻辑生成pos neg 比例

In [2]:
def changeSplitTag(row):
    row=eval(row)
    prepath ='/home/hdj/bug_localization/data/SWT/clean/' if 'swt' in row[0] else '/home/hdj/bug_localization/data/ZXing/clean/'
    res=[]
    for path in row:
        #先默认按照最后一个.代表文件，去查找如果不在，取倒数第二个点
        index=path.rindex('.')
        head=path[:index]
        tail=path[index:]
        head=head.replace('.','/')
        path=head+tail
        if not os.path.exists(prepath+path):
            print('该文件可能处理有问题 : ',prepath+path)
        res.append(path)
    return res


#取得fixed——file 不带前缀地址
def pathNorm(src_addressed,reg):
    res=[]
    for a in src_addressed:
        index = a.index(reg)
        path = a[index:]
        # path = '.'.join(path.split('/'))
        res.append(path)
    return res

def train_generate(prepath, project_name, more,choosed_path,sample=1000):
    if more:
        report = pd.read_csv(prepath + project_name + 'train_and_test.csv')

    else:
        report = pd.read_csv(prepath + project_name + 'train_only.csv')
    # report['fixed_files']=report['fixed_files'].astype(list)
    report_ids = list(report['bug_id'])
    fixed_files_str = list(report['fixed_files'])
    fixed_files = []

    # 将string转为list
    for fixed in fixed_files_str:
        # print(fixed)
        # print(type(eval(fixed)),len(fixed))
        # 过滤为空的文件 还得过滤解析AST失败的文件
        fixed = eval(fixed)
        # print('过滤前 fided file :',fixed)

        fixed = [file for file in fixed if file in choosed_path]
        # print('过滤后 fided file :', fixed)
        fixed_files.append(fixed)
    # 转换结束
    # break
    out_ids = []
    out_neg_ids=[]
    out_pos_path = []
    out_neg_path = []
    _id=None
    for id, fixed_list in zip(report_ids, fixed_files):
        for fixed in fixed_list:
            _id=id
            out_ids.append(id)
            out_pos_path.append(fixed)

            # Getting the list of source files recursively from the source directory
        Neg_files = set(choosed_path) - set(fixed_list)
        A_P_len = len(fixed_list)
        print('A_P_len ', A_P_len,'sample :',sample-A_P_len)
        #TODO 改成取样1000-A_P_len
        sample_num=sample-A_P_len
        if sample_num>len(Neg_files):
            sample_num=len(Neg_files)
        #EDIT 采样改为1:1**************
        sample_num=A_P_len
        Neg_files_k = random.sample(Neg_files, k=sample_num)
        for neg_file in Neg_files_k:
            out_neg_ids.append(_id)
            out_neg_path.append(neg_file)
    train_pos = pd.DataFrame()
    train_neg = pd.DataFrame()
    train_pos['id'] = out_ids
    train_pos['path'] = out_pos_path
    train_pos['label'] = 1

    train_neg['id'] = out_neg_ids
    train_neg['path'] = out_neg_path
    train_neg['label'] = 0
    print(len(out_ids), len(out_pos_path), len(out_neg_path))
    if more:
        train_pos.to_csv(prepath + project_name + '80_training_pos.csv', index=False)
        train_neg.to_csv(prepath + project_name + '80_training_neg.csv', index=False)
    else:
        train_pos.to_csv(prepath + project_name + '20_training_pos.csv', index=False)
        train_neg.to_csv(prepath + project_name + '20_training_neg.csv', index=False)
    #源码目录 这里选取ast和token文件都存在的修复
def readAll(path):
    fixed_files=set()
    with open(path,'r',encoding='utf-8') as f_in:
        for line in f_in:
            line=line.strip()
            fixed_files.add(line)
    return fixed_files

# 将每个数据集分割成80 20%两部分 aspectj_train_and_test.csv aspectj_train_only.csv

In [3]:
#分割数据集为train val test
prepath='/data/hdj/cross_project_trans/report/'
project_names=['aspectj_','swt_','zxing_']

srcs={'/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/':'org.aspectj/modules','/home/hdj/bug_localization/data/SWT/clean/org/eclipse/swt':'org/eclipse/swt','/home/hdj/bug_localization/data/ZXing/':'com/google/zxing'}
i=0
# test_paths=['/data/hdj/cross_project_trans/report/aspectj_test','/data/hdj/cross_project_trans/report/swt_test','/data/hdj/cross_project_trans/report/zxing_test']
test_paths=['/data/hdj/data/CodeBERT/data/codesearch/test/aspectj_test','/data/hdj/data/CodeBERT/data/codesearch/test/swt_test','/data/hdj/data/CodeBERT/data/codesearch/test/zxing_test']
filter_path='/data/hdj/cross_project_trans/report/fixed_files'
# for project_name in project_names:
#     report=pd.read_csv(prepath+project_name+'report_clean.csv')#.sample(frac=1)
#     if project_name!='aspectj_':
#         report['fixed_files']=report['fixed_files'].apply(changeSplitTag)

#     #TODO 到时这里做打散操作
#     train_len=int(report.shape[0] * 0.8)
#     train=report.loc[:train_len-1,]
#     test=report.loc[train_len:,]

#     print('report shape : ',report.shape,'train len: ',train_len,)
#     print('train shape : ',train.shape,'test shape : ',test.shape)
#     print(type(train),type(test))
#     train.to_csv(prepath+project_name+'train_and_test.csv',index=False)
#     test.to_csv(prepath+project_name+'train_only.csv',index=False)

# 构造train逻辑上数据aspectj_80_training_pos.csv aspectj_80_training_neg.csv

In [4]:

for src,reg in  srcs.items():
    print('当前处理的项目 ： ',reg)
    # src_addresses = glob.glob(str(src) + '/**/*.java', recursive=True) #文件不为空
    # src_addresses = glob.glob(str('/home/hdj/bug_localization/data/SWT/clean/src/org/eclipse/swt') + '/**/*.java', recursive=True) #文件不为空

    # src_addresses=pathNorm(src_addresses,reg)
    #TODO 等tree_blocks生成好了改路径
    # ast_success= pd.read_pickle('/home/hdj/cross_project_trans/out/'+project_names[i]+'tree_blocks.pkl')
    # ast_success_paths = list(ast_success['path'])
    # print('ast len : ',len(ast_success_paths),' src len: ',len(src_addresses))
    # print(reg,'查看两个地址示例',ast_success_paths[0],' **** ',src_addresses[0])
    # choosed_path=set(ast_success_paths) & set(src_addresses)
    # print('token不为空大小 : ',len(src_addresses),' ast不为空大小 : ',len(ast_success_paths),' ast 和 token不为空交集的大小: ',len(choosed_path))

    choosed_path=readAll(os.path.join(filter_path,project_names[i]+'filter.txt'))
    # print(src_addresses[0])
    # #读取源码结束
    #
    #生成80% train
    train_generate(prepath,project_names[i],True,choosed_path,sample=100)
    #生成80%train结束

    #生成20% train
    train_generate(prepath,project_names[i],False,choosed_path,sample=100)
    # 生成20% train结束
    i+=1

当前处理的项目 ：  org.aspectj/modules
A_P_len  1 sample : 99
A_P_len  1 sample : 99
A_P_len  3 sample : 97
A_P_len  6 sample : 94
A_P_len  4 sample : 96
A_P_len  2 sample : 98
A_P_len  3 sample : 97
A_P_len  2 sample : 98
A_P_len  4 sample : 96
A_P_len  1 sample : 99
A_P_len  3 sample : 97
A_P_len  3 sample : 97
A_P_len  5 sample : 95
A_P_len  2 sample : 98
A_P_len  2 sample : 98
A_P_len  2 sample : 98
A_P_len  3 sample : 97
A_P_len  2 sample : 98
A_P_len  3 sample : 97
A_P_len  5 sample : 95
A_P_len  2 sample : 98
A_P_len  2 sample : 98
A_P_len  3 sample : 97
A_P_len  6 sample : 94
A_P_len  2 sample : 98
A_P_len  3 sample : 97
A_P_len  1 sample : 99
A_P_len  1 sample : 99
A_P_len  1 sample : 99
A_P_len  1 sample : 99
A_P_len  20 sample : 80
A_P_len  1 sample : 99
A_P_len  1 sample : 99
A_P_len  1 sample : 99
A_P_len  1 sample : 99
A_P_len  4 sample : 96
A_P_len  4 sample : 96
A_P_len  11 sample : 89
A_P_len  4 sample : 96
A_P_len  17 sample : 83
A_P_len  4 sample : 96
A_P_len  1 sample : 99


In [16]:
# choosed_path

# 生成test 逻辑上的数据 ，基本不需要改变
/data/hdj/data/CodeBERT/data/codesearch/test/zxing_test/test0.csv

In [8]:
i=0
for src,reg in  srcs.items():
    print('当前处理的项目22 ： ',reg)
    #生成test开始
    choosed_path=readAll(os.path.join(filter_path,project_names[i]+'filter.txt'))
    test=pd.read_csv(prepath+project_names[i]+'train_only.csv')
#     test=pd.read_csv(prepath+project_names[i]+'train_and_test.csv')
    test_ids=list(test['bug_id'])
    test_fixed_files_str=list(test['fixed_files'])
    test_fixed_files=[]
    #去除文件为空
    for fixed in test_fixed_files_str:
        # print(fixed)
        # print(type(eval(fixed)),len(fixed))
        #过滤为空的文件
        fixed=eval(fixed)
        fixed=[file for file in fixed if file in choosed_path]
        test_fixed_files.append(fixed)
    #去除完成
#     print(test_fixed_files[0],type(test_fixed_files[0]),type(test_ids[0]))
    print('project name : ',project_names[i],'test 集大小 :',len(test_ids),len(test_fixed_files))
#     total=1000
    total=300
    j=0
    for id,fixed_files in zip(test_ids,test_fixed_files):
        out_ids=[]
        out_path=[]
        out_label=[]
        # print('j : ',j)
        neg_sample=total-len(fixed_files)
        Neg_files = set(choosed_path) - set(fixed_files)
        if neg_sample>len(Neg_files):
            neg_sample=len(Neg_files)
        Neg_files_k = random.sample(Neg_files, k=neg_sample)
        if len(fixed_files)==0:
            print("test 的fixed files数量为0 :",id)
            continue
        for fixed in fixed_files:
            # f_out.write()
            out_ids.append(id)
            out_path.append(fixed)
            out_label.append(1)
        for unfixed in Neg_files_k:
            out_ids.append(id)
            out_path.append(unfixed)
            out_label.append(0)

        out_data=pd.DataFrame()
        out_data['id']=out_ids
        out_data['path']=out_path
        out_data['label']=out_label
        out_data.to_csv(os.path.join(test_paths[i],'test'+str(j)+'.csv') ,index=False)
        j+=1
    i+=1
#     break
    #生成tes结束


当前处理的项目22 ：  org.aspectj/modules
project name :  aspectj_ test 集大小 : 58 58
当前处理的项目22 ：  org/eclipse/swt
project name :  swt_ test 集大小 : 20 20
当前处理的项目22 ：  com/google/zxing
project name :  zxing_ test 集大小 : 4 4


# code_class_method_name 和report_pos_summary_description信息解析

In [1]:
class ReportPreprocessing:
    """Class to preprocess bug reports"""
    
    __slots__ = ['bug_reports']
    
    def __init__(self, bug_reports):
        self.bug_reports = bug_reports
    
    def extract_stack_traces(self):
        """Extracting stack traces from bug reports"""
        
        # Simple pattern to retrieve stack traces
        pattern = re.compile(r' at (.*?)\((.*?)\)')
        
        # Signs of a true stack trace to check in the retrieved regex grouping
        signs = ['.java', 'Unknown Source', 'Native Method']

        for report in self.bug_reports.values():
            st_candid = re.findall(pattern, report.description)
            
            # Filter actual stack traces from retrieved candidates
            st = [x for x in st_candid if any(s in x[1] for s in signs)]
            report.stack_traces = st
    
    def pos_tagging(self):
        """Extracing specific pos tags from bug reports' summary and description"""
        
        for report in self.bug_reports.values():
            
            # Tokenizing using word_tokeize for more accurate pos-tagging
            summary = nltk.wordpunct_tokenize(report.summary)
            description = nltk.wordpunct_tokenize(report.description)
            # summ_tok = nltk.word_tokenize(report.summary)
            # print(type(summary))
            # print(type(description))
            summ_tok = nltk.word_tokenize(' '.join(summary))
            desc_tok = nltk.word_tokenize(' '.join(description))
            # desc_tok = nltk.word_tokenize(report.description)
            sum_pos = nltk.pos_tag(summ_tok)
            desc_pos = nltk.pos_tag(desc_tok)
            
            report.pos_tagged_summary = [token for token, pos in sum_pos
                                         if 'NN' in pos or 'VB' in pos]
            report.pos_tagged_description = [token for token, pos in desc_pos
                                             if 'NN' in pos or 'VB' in pos]
        
    def tokenize(self):
        """Tokenizing bug reports into tokens"""
        
        for report in self.bug_reports.values():
            report.summary = nltk.wordpunct_tokenize(report.summary)
            report.description = nltk.wordpunct_tokenize(report.description)
    
    def _split_camelcase(self, tokens):
    
        # Copy tokens
        returning_tokens = tokens[:]
        
        for token in tokens:
            split_tokens = re.split(fr'[{string.punctuation}]+', token)
            
            # If token is split into some other tokens
            if len(split_tokens) > 1:
                returning_tokens.remove(token)
                # Camel case detection for new tokens
                for st in split_tokens:
                    camel_split = inflection.underscore(st).split('_')
                    if len(camel_split) > 1:
                        returning_tokens.append(st)
                        returning_tokens += camel_split
                    else:
                        returning_tokens.append(st)
            else:
                camel_split = inflection.underscore(token).split('_')
                if len(camel_split) > 1:
                    returning_tokens += camel_split
    
        return returning_tokens
    
    def split_camelcase(self):
        """Split CamelCase identifiers"""
        
        for report in self.bug_reports.values():
            report.summary = self._split_camelcase(report.summary)
            report.description = self._split_camelcase(report.description)
            report.pos_tagged_summary = self._split_camelcase(report.pos_tagged_summary)
            report.pos_tagged_description = self._split_camelcase(report.pos_tagged_description)
            
    def normalize(self):
        """Removing punctuation, numbers and also lowercase conversion"""
        
        # Building a translate table for punctuation and number removal
        punctnum_table = str.maketrans({c : None for c in string.punctuation + string.digits})
        
        for report in self.bug_reports.values():
            summary_punctnum_rem = [token.translate(punctnum_table)
                                    for token in report.summary]
            desc_punctnum_rem = [token.translate(punctnum_table)
                                 for token in report.description]
            pos_sum_punctnum_rem = [token.translate(punctnum_table)
                                  for token in report.pos_tagged_summary]
            pos_desc_punctnum_rem = [token.translate(punctnum_table)
                                  for token in report.pos_tagged_description]
            
            report.summary = [token.lower() for token
                              in summary_punctnum_rem if token]
            report.description = [token.lower() for token
                                  in desc_punctnum_rem if token]
            report.pos_tagged_summary = [token.lower() for token
                                         in pos_sum_punctnum_rem if token]
            report.pos_tagged_description = [token.lower() for token
                                             in pos_desc_punctnum_rem if token]
            
    def remove_stopwords(self):
        """Removing stop words from tokens"""
        
        for report in self.bug_reports.values():
            report.summary = [token for token in report.summary
                              if token not in stop_words]
            report.description = [token for token in report.description
                                  if token not in stop_words]
            report.pos_tagged_summary = [token for token in report.pos_tagged_summary
                                         if token not in stop_words]
            report.pos_tagged_description = [token for token in report.pos_tagged_description
                                             if token not in stop_words]
            
    def remove_java_keywords(self):
        """Removing Java language keywords from tokens"""

        for report in self.bug_reports.values():
            report.summary = [token for token in report.summary
                              if token not in java_keywords]
            report.description = [token for token in report.description
                                  if token not in java_keywords]
            report.pos_tagged_summary = [token for token in report.pos_tagged_summary
                                         if token not in java_keywords]
            report.pos_tagged_description = [token for token in report.pos_tagged_description
                                             if token not in java_keywords]
    def remove_less_k_or_not_alpha(self):
        for report in self.bug_reports.values():
            report.summary = [token for token in report.summary if token.isalnum() and len(token)>=3]
            report.description = [token for token in report.description if token.isalnum() and len(token)>=3]
            report.pos_tagged_summary = [token for token in report.pos_tagged_summary if token.isalnum() and len(token)>=3]
            report.pos_tagged_description = [token for token in report.pos_tagged_description if token.isalnum() and len(token)>=3]
    def stem(self):
        """Stemming tokens"""
        
        # Stemmer instance
        stemmer = PorterStemmer()

        for report in self.bug_reports.values():
            report.summary = dict(zip(['stemmed', 'unstemmed'],
                                      [[stemmer.stem(token) for token in report.summary],
                                       report.summary]))
            
            report.description = dict(zip(['stemmed', 'unstemmed'],
                                          [[stemmer.stem(token) for token in report.description],
                                           report.description]))
            
            report.pos_tagged_summary = dict(zip(['stemmed', 'unstemmed'],
                                                 [[stemmer.stem(token) for token
                                                   in report.pos_tagged_summary],
                                                  report.pos_tagged_summary]))
            
            report.pos_tagged_description = dict(zip(['stemmed', 'unstemmed'],
                                                     [[stemmer.stem(token) for token
                                                       in report.pos_tagged_description],
                                                      report.pos_tagged_description]))

    def preprocess(self):
        """Run the preprocessing"""
        
        self.extract_stack_traces()
        self.pos_tagging()
        self.tokenize()
        # self.split_camelcase()
        # self.normalize()
        self.remove_stopwords()
        self.remove_java_keywords()
        # self.stem()
        self.remove_less_k_or_not_alpha()

class SrcPreprocessing:
    """Class to preprocess source codes"""
    
    __slots__ = ['src_files']
    
    def __init__(self, src_files):
        self.src_files = src_files
    
    def pos_tagging(self):
        """Extracing specific pos tags from comments"""
        
        for src in self.src_files.values():
            
            # Tokenizing using word_tokeize for more accurate pos-tagging
            comments = nltk.wordpunct_tokenize(src.comments)
            comments_tok = nltk.word_tokenize(' '.join(comments))
            # comments_tok = nltk.word_tokenize(src.comments)
            comments_pos = nltk.pos_tag(comments_tok)
             
            src.pos_tagged_comments = [token for token, pos in comments_pos
                                       if 'NN' in pos or 'VB' in pos]
    
    def tokenize(self):
        """Tokenizing source codes into tokens"""
        
        for src in self.src_files.values():
            src.all_content = nltk.wordpunct_tokenize(src.all_content)
            src.comments = nltk.wordpunct_tokenize(src.comments)
    
    def _split_camelcase(self, tokens):
    
        # Copy tokens
        returning_tokens = tokens[:]
        
        for token in tokens:
            split_tokens = re.split(fr'[{string.punctuation}]+', token)
            
            # If token is split into some other tokens
            if len(split_tokens) > 1:
                returning_tokens.remove(token)
                # Camel case detection for new tokens
                for st in split_tokens:
                    camel_split = inflection.underscore(st).split('_')
                    if len(camel_split) > 1:
                        returning_tokens.append(st)
                        returning_tokens += camel_split
                    else:
                        returning_tokens.append(st)
            else:
                camel_split = inflection.underscore(token).split('_')
                if len(camel_split) > 1:
                    returning_tokens += camel_split
    
        return returning_tokens
    
    def split_camelcase(self):
        """Split CamelCase identifiers"""
        
        for src in self.src_files.values():
            src.all_content = self._split_camelcase(src.all_content)
            src.comments = self._split_camelcase(src.comments)
            src.class_names = self._split_camelcase(src.class_names)
            src.attributes = self._split_camelcase(src.attributes)
            src.method_names = self._split_camelcase(src.method_names)
            src.variables = self._split_camelcase(src.variables)
            src.file_name = self._split_camelcase(src.file_name)
            src.pos_tagged_comments = self._split_camelcase(src.pos_tagged_comments)
            
    def normalize(self):
        """Removing punctuation, numbers and also lowercase conversion"""
        
        # Building a translate table for punctuation and number removal
        punctnum_table = str.maketrans({c : None for c in string.punctuation + string.digits})
        
        for src in self.src_files.values():
            content_punctnum_rem = [token.translate(punctnum_table)
                                    for token in src.all_content]
            comments_punctnum_rem = [token.translate(punctnum_table)
                                     for token in src.comments]
            classnames_punctnum_rem = [token.translate(punctnum_table)
                                       for token in src.class_names]
            attributes_punctnum_rem = [token.translate(punctnum_table)
                                       for token in src.attributes]
            methodnames_punctnum_rem = [token.translate(punctnum_table)
                                        for token in src.method_names]
            variables_punctnum_rem = [token.translate(punctnum_table)
                                      for token in src.variables]
            filename_punctnum_rem = [token.translate(punctnum_table)
                                     for token in src.file_name]
            pos_comments_punctnum_rem = [token.translate(punctnum_table)
                                         for token in src.pos_tagged_comments]
            
            src.all_content = [token.lower() for token
                               in content_punctnum_rem if token]
            src.comments = [token.lower() for token
                            in comments_punctnum_rem if token]
            src.class_names = [token.lower() for token
                               in classnames_punctnum_rem if token]
            src.attributes = [token.lower() for token
                              in attributes_punctnum_rem if token]
            src.method_names = [token.lower() for token
                                in methodnames_punctnum_rem if token]
            src.variables = [token.lower() for token
                             in variables_punctnum_rem if token]
            src.file_name = [token.lower() for token
                             in filename_punctnum_rem if token]
            src.pos_tagged_comments = [token.lower() for token
                                       in pos_comments_punctnum_rem if token]
            
    def remove_stopwords(self):
        """Removing stop words from tokens"""
        
        for src in self.src_files.values():
            src.all_content = [token for token in src.all_content
                               if token not in stop_words]
            src.comments = [token for token in src.comments
                            if token not in stop_words]
            src.class_names = [token for token in src.class_names
                               if token not in stop_words]
            src.attributes = [token for token in src.attributes
                              if token not in stop_words]
            src.method_names = [token for token in src.method_names
                                if token not in stop_words]
            src.variables = [token for token in src.variables
                             if token not in stop_words]
            src.file_name = [token for token in src.file_name
                             if token not in stop_words]
            src.pos_tagged_comments = [token for token in src.pos_tagged_comments
                                       if token not in stop_words]
            
    def remove_java_keywords(self):
        """Removing Java language keywords from tokens"""

        for src in self.src_files.values():
            src.all_content = [token for token in src.all_content
                               if token not in java_keywords]
            src.comments = [token for token in src.comments
                            if token not in java_keywords]
            src.class_names = [token for token in src.class_names
                               if token not in java_keywords]
            src.attributes = [token for token in src.attributes
                              if token not in java_keywords]
            src.method_names = [token for token in src.method_names
                                if token not in java_keywords]
            src.variables = [token for token in src.variables
                             if token not in java_keywords]
            src.file_name = [token for token in src.file_name
                             if token not in java_keywords]
            src.pos_tagged_comments = [token for token in src.pos_tagged_comments
                                       if token not in java_keywords]
    def remove_less_k_or_not_alpha(self):
        for src in self.src_files.values():
            src.all_content = [token for token in src.all_content if token.isalnum() and len(token) >= 3]
            src.comments = [token for token in src.comments if token.isalnum() and len(token) >= 3]
            src.class_names = [token for token in src.class_names if token.isalnum() and len(token) >= 3]
            src.attributes = [token for token in src.attributes if token.isalnum() and len(token) >= 3]
            src.method_names = [token for token in src.method_names if token.isalnum() and len(token) >= 3]
            src.file_name = [token for token in src.file_name if token.isalnum() and len(token) >= 3]
            src.pos_tagged_comments = [token for token in src.pos_tagged_comments if token.isalnum() and len(token) >= 3]
    def stem(self):
        """Stemming tokens"""
        
        # Stemmer instance
        stemmer = PorterStemmer()
        
        for src in self.src_files.values():
            src.all_content = dict(zip(['stemmed', 'unstemmed'],
                                       [[stemmer.stem(token) for token in src.all_content],
                                        src.all_content]))
            
            src.comments = dict(zip(['stemmed', 'unstemmed'],
                                    [[stemmer.stem(token) for token in src.comments],
                                     src.comments]))
            
            src.class_names = dict(zip(['stemmed', 'unstemmed'],
                                       [[stemmer.stem(token) for token in src.class_names],
                                        src.class_names]))
            
            src.attributes = dict(zip(['stemmed', 'unstemmed'],
                                      [[stemmer.stem(token) for token in src.attributes],
                                       src.attributes]))
            
            src.method_names = dict(zip(['stemmed', 'unstemmed'],
                                        [[stemmer.stem(token) for token in src.method_names],
                                         src.method_names]))
            
            src.variables = dict(zip(['stemmed', 'unstemmed'],
                                     [[stemmer.stem(token) for token in src.variables],
                                      src.variables]))
            
            src.file_name = dict(zip(['stemmed', 'unstemmed'],
                                     [[stemmer.stem(token) for token in src.file_name],
                                      src.file_name]))
            
            src.pos_tagged_comments = dict(zip(['stemmed', 'unstemmed'],
                                      [[stemmer.stem(token) for token in src.pos_tagged_comments],
                                       src.pos_tagged_comments]))

    def preprocess(self):
        """Run the preprocessing"""
        
        self.pos_tagging()
        self.tokenize()
        # self.split_camelcase()
        # self.normalize()
        self.remove_stopwords()
        self.remove_java_keywords()
        # self.stem()
        self.remove_less_k_or_not_alpha()

In [9]:
# Dataset root directory
# _DATASET_ROOT = Path('D:\PythonWorkSet\\bug-localization-master\data')
_DATASET_ROOT = Path('/home/hdj/bug_localization/data')

Dataset = namedtuple('Dataset', ['name', 'root', 'src', 'bug_repo'])

# Source codes and bug repositories
aspectj = Dataset(
    'aspectj',
    _DATASET_ROOT / 'AspectJ',
    _DATASET_ROOT / 'AspectJ/AspectJ-1.5',
    _DATASET_ROOT / 'AspectJ/AspectJBugRepository.xml'
)

swt = Dataset(
    'swt',
    _DATASET_ROOT / 'SWT',
    _DATASET_ROOT / 'SWT/SWT-3.1',
    _DATASET_ROOT / 'SWT/SWTBugRepository.xml'
)

zxing = Dataset(
    'zxing',
    _DATASET_ROOT / 'ZXing',
    _DATASET_ROOT / 'ZXing/ZXing-1.6',
    _DATASET_ROOT / 'ZXing/ZXingBugRepository.xml'
)

### Current dataset in use. (change this name to change the dataset)

DATASET = aspectj
# DATASET = swt
# DATASET = zxing

In [6]:
# for key, value in data.items():
#     # vals.append(value.all_content)
#     i += 1
#     print(type(value.class_names),value.class_names)
# #     class_method_names=value.class_names+value.method_names
# #     if (len(class_method_names) == 0):
# #         nodata += 1
#         # print(key)
# #         continue
# #     codeDict[key]=' '.join(class_method_names[:15])
#     # codeDict[key]=' '.join(class_method_names)
# #     codeLen.append(len(class_method_names))
# #     print(key, value.all_content,value.comments,value.class_names,value.attributes,value.method_names,
# #           value.variables,value.file_name,value.exact_file_name,value.package_name,value.pos_tagged_comments)
#     print(key)
#     print('all_content :',value.all_content)  # string public class 开始至最后
#     print('*' * 10)
#     print('comments :',value.comments)  # 类型 string
#     print('*' * 10)
#     print('class_names :',value.class_names)  # 类型 list ['Util', 'Constants', 'OSGIBundle', 'RequiredBundle']
#     print('*' * 10)
#     print('attributes :',value.attributes)  # list
#     # ['TESTSRC', 'JAVA5_SRC', 'JAVA5_TESTSRC', 'JAVA5_VM', 'BUNDLE_NAME', 'BUNDLE_SYMBOLIC_NAME', 'BUNDLE_VERSION', 'BUNDLE_ACTIVATOR', 'BU
#     print('*' * 10)
#     print('method_names :',value.method_names)  # list
#     # ['shortVersion', 'replace', 'visitFiles', 'iaxIfNotCanReadDir', 'iaxIfNotCanReadFile', 'iaxIfNotCanWriteDir', 'iaxIfNull', 'renderExce
#     print('*' * 10)
#     print('variables :',value.variables)  # list
#     # ['java5VM', 'loc', 'result', 'start', 'files', 'passed', 'i', 'sw', 'pw', 'sb', 'i', 'files', 'i', 'tempFile', 'i', 'result', 'hits', 
#     print('*' * 10)
#     print('file_name :',value.file_name)  # list ['Util']
#     print('*' * 10)
#     print('exact_file_name :',value.exact_file_name)  # string Util
#     print('*' * 10)
#     print('package_name :',value.package_name)  # string org.aspectj.internal.tools.build
#     print('*' * 10)
#     print('pos_tagged_comments :',value.pos_tagged_comments)
#     print('*' * 10)
#     if i == 2:
#             break


# 将code和report都解析出来 并经过一定的预处理生成

In [4]:
DATASET = zxing
def main():
    #Parser 做两件事 解析report 和code
    parser = Parser(DATASET)
    #src_parser 返回dict {'code_file':SourceFile}
    '''
     SourceFile
     def __init__(self, all_content, comments, class_names, attributes,
                 method_names, variables, file_name, package_name):
        self.all_content = all_content
        self.comments = comments
        self.class_names = class_names
        self.attributes = attributes
        self.method_names = method_names
        self.variables = variables
        self.file_name = file_name
        self.exact_file_name = file_name[0]
        self.package_name = package_name
        self.pos_tagged_comments = None
    '''
    src_prep = SrcPreprocessing(parser.src_parser())
    src_prep.preprocess()
    #保存为dict格式 {'code_path':SourceFile 对象}
    with open(DATASET.root / 'preprocessed_src.pickle', 'wb') as file:
        pickle.dump(src_prep.src_files, file, protocol=pickle.HIGHEST_PROTOCOL)
    
    #report_parser 返回的是dict {'report_id':BugReport} BugReport()
    '''
    BugReport
         def __init__(self, summary, description, fixed_files):
            self.summary = summary
            self.description = description
            self.fixed_files = fixed_files
            self.pos_tagged_summary = None
            self.pos_tagged_description = None
            self.stack_traces = None
    '''
    report_prep = ReportPreprocessing(parser.report_parser())
    report_prep.preprocess()
    #保存的dict格式 {'report_id':BugReport对象}
    with open(DATASET.root / 'preprocessed_reports.pickle', 'wb') as file:
        pickle.dump(report_prep.bug_reports, file, protocol=pickle.HIGHEST_PROTOCOL)
main()

here
<class 'collections.OrderedDict'>
here
<class 'collections.OrderedDict'>


# 这里有选择的组合一部分信息单独保存起来

In [5]:
if __name__ == '__main__':
    # main()
    #拼接成地址 /home/hdj/bug_localization/data/ZXing/preprocessed_src.pickle
    f = open(DATASET.root / 'preprocessed_src.pickle', 'rb')
    data = pickle.load(f)
    dataFrame = pd.DataFrame()
    codeDict=OrderedDict()#用dict格式来保存code的信息 
    i = 0
    codeLen=[]#统计长度信息
    nodata=0
    for key, value in data.items():
        # vals.append(value.all_content)
        i += 1
#         print(type(value.class_names),value.class_names)
        class_method_names=value.class_names+value.method_names
        if (len(class_method_names) == 0):
            nodata += 1
            # print(key)
            continue
        codeDict[key]=' '.join(class_method_names[:15])
        # codeDict[key]=' '.join(class_method_names)
        codeLen.append(len(class_method_names))

        # print(key, value.all_content,value.comments,value.class_names,value.attributes,value.method_names,
        #       value.variables,value.file_name,value.exact_file_name,value.package_name,value.pos_tagged_comments)
        # print(key)
        # print('all_content :',value.all_content)  # string public class 开始至最后
        # print('*' * 10)
        # print('comments :',value.comments)  # 类型 string
        # print('*' * 10)
        # print('class_names :',value.class_names)  # 类型 list ['Util', 'Constants', 'OSGIBundle', 'RequiredBundle']
        # print('*' * 10)
        # print('attributes :',value.attributes)  # list
        # # ['TESTSRC', 'JAVA5_SRC', 'JAVA5_TESTSRC', 'JAVA5_VM', 'BUNDLE_NAME', 'BUNDLE_SYMBOLIC_NAME', 'BUNDLE_VERSION', 'BUNDLE_ACTIVATOR', 'BUNDLE_VENDOR', 'REQUIRE_BUNDLE', 'IMPORT_PACKAGE', 'BUNDLE_CLASSPATH', 'NAMES', 'manifest', 'attributes', 'text', 'name', 'versions', 'optional']
        # print('*' * 10)
        # print('method_names :',value.method_names)  # list
        # # ['shortVersion', 'replace', 'visitFiles', 'iaxIfNotCanReadDir', 'iaxIfNotCanReadFile', 'iaxIfNotCanWriteDir', 'iaxIfNull', 'renderException', 'canWriteDir', 'path', 'path', 'canReadDir', 'canReadFile', 'delete', 'deleteContents', 'makeTempDir', 'close', 'isEmpty', 'closeSilently', 'closeSilently', 'reportMemberDiffs', 'copy', 'OSGIBundle', 'getAttribute', 'getClasspath', 'getRequiredBundles', 'RequiredBundle']
        # print('*' * 10)
        # print('variables :',value.variables)  # list
        # # ['java5VM', 'loc', 'result', 'start', 'files', 'passed', 'i', 'sw', 'pw', 'sb', 'i', 'files', 'i', 'tempFile', 'i', 'result', 'hits', 'i', 'curHit', 'j', 'prefix', 'i', 'i', 'result', 'names', 'cp', 'st', 'result', 'i', 'value', 'st', 'result', 'i', 'skips', 'token', 'first', 'patch', 'st', 'vers', 'opt', 'RESOLUTION', 'VERSION', 'token', 'start', 'end', 'start', 'end']
        # print('*' * 10)
        # print('file_name :',value.file_name)  # list ['Util']
        # print('*' * 10)
        # print('exact_file_name :',value.exact_file_name)  # string Util
        # print('*' * 10)
        # print('package_name :',value.package_name)  # string org.aspectj.internal.tools.build
        # print('*' * 10)
        # print('pos_tagged_comments :',value.pos_tagged_comments)
        # print('*' * 10)
        #
        # break
        # if i == 2:
        #     break
    codeLen = sorted(codeLen)
    print('总共文件数 :',len(codeLen),' 长度为0的文件数: ',nodata,codeLen)
    a = np.array(codeLen)
    # 25%分位数
    print('code 的 25%', np.percentile(a, 10))
    # 75%分位数
    print(np.percentile(a, 20))
    print(np.percentile(a, 40))
    print('中位数 :', np.median(a))
    print(np.percentile(a, 60))
    print(np.percentile(a, 80))
    print(np.percentile(a, 90))

    #/home/hdj/bug_localization/data/ZXing/preprocessed_reports.pickle
    f = open(DATASET.root / 'preprocessed_reports.pickle', 'rb')
    x = pickle.load(f)
    i = 0
    reportDict=OrderedDict()
    reportLen=[]

    for key, value in x.items():
        i += 1
        summ_descrip = []
        reportSet = set()  # 去重使用 这里还将重复的词都给去掉了
        for token in value.pos_tagged_summary+value.pos_tagged_description:
            if token not in reportSet:
                summ_descrip.append(token)
                reportSet.add(token)
        reportDict[key]=' '.join(summ_descrip[:100])
        # reportDict=' '.join(summ_descrip)
        reportLen.append(len(summ_descrip))
    reportLen=sorted(reportLen)
    print('总共文件数 :', len(reportLen), reportLen)
    a = np.array(reportLen)
    # 25%分位数
    print('report 的 10%',np.percentile(a, 10))
    # 75%分位数
    print('20 :',np.percentile(a, 20))
    print('40 :',np.percentile(a, 40))
    # 中位数
    print('中位数 :',np.median(a))
    print('60 :',np.percentile(a, 60))
    print('80 :',np.percentile(a, 80))
    print('90 :',np.percentile(a, 90))

    #将两个字典进行保存
    out_path='/data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values'
    # project='aspectj_'
    # project='swt_'
    project='zxing_'
    #拼接成 /data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/zxing_code_class_method_names.pickle
    with open(os.path.join(out_path,project+'code_class_method_names') , 'wb') as file:
        pickle.dump(codeDict,file, protocol=pickle.HIGHEST_PROTOCOL)
    #拼接成 /data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/zxing_report_pos_summ_descr.pickle
    with open(os.path.join(out_path, project+'report_pos_summ_descr.pickle'), 'wb') as file:
        pickle.dump(reportDict, file, protocol=pickle.HIGHEST_PROTOCOL)
        # print(key,value.summary,value.description,value.fixed_files)
        # print(key)
        # print('summary :',value.summary)
        # print('*' * 10)
        # print('description :',value.description)
        # print('*' * 10)
        # print('fixed_files :',value.fixed_files)
        # print('*' * 10)
        # print('pos_tagged_summary :',value.pos_tagged_summary)
        # print('*' * 10)
        # print('pos_tagged_description :',value.pos_tagged_description)
        # print('*' * 10)
        # print('stack_traces :',value.stack_traces)
        # print('*' * 10)
        # break
        # if i == 2:
        #     break

<class 'list'> ['QRCapturedJournalListener']
<class 'list'> ['Camera']
<class 'list'> ['ZXingUiApplication']
<class 'list'> ['AppPermissionsManager']
<class 'list'> ['HistoryScreen', 'ButtonListener']
<class 'list'> ['ZXingLMMainScreen', 'ButtonListener', 'FileConnectionThread']
<class 'list'> ['AboutScreen', 'ButtonListener']
<class 'list'> ['SettingsScreen', 'ButtonListener']
<class 'list'> ['HelpScreen', 'ButtonListener']
<class 'list'> ['ReasonableTimer']
<class 'list'> ['URLDecoder']
<class 'list'> ['Log']
<class 'list'> ['AppSettings']
<class 'list'> ['DecodeHistory']
<class 'list'> ['DecodeHistoryItem']
<class 'list'> ['IntentIntegrator']
<class 'list'> ['IntentResult']
<class 'list'> ['SmsAddressGenerator']
<class 'list'> ['EmailGenerator']
<class 'list'> ['WifiGenerator']
<class 'list'> ['Generator']
<class 'list'> ['StylesDefs']
<class 'list'> ['GeneratorException']
<class 'list'> ['StringConstants']
<class 'list'> ['UrlGenerator']
<class 'list'> ['TimeZoneList', 'TimeZoneInf

<class 'list'> ['QRCapturedJournalListener']
<class 'list'> ['Camera']
<class 'list'> ['ZXingUiApplication']
<class 'list'> ['AppPermissionsManager']
<class 'list'> ['HistoryScreen', 'ButtonListener']
<class 'list'> ['ZXingLMMainScreen', 'ButtonListener', 'FileConnectionThread']
<class 'list'> ['AboutScreen', 'ButtonListener']
<class 'list'> ['SettingsScreen', 'ButtonListener']
<class 'list'> ['HelpScreen', 'ButtonListener']
<class 'list'> ['ReasonableTimer']
<class 'list'> ['URLDecoder']
<class 'list'> ['Log']
<class 'list'> ['AppSettings']
<class 'list'> ['DecodeHistory']
<class 'list'> ['DecodeHistoryItem']
<class 'list'> ['IntentIntegrator']
<class 'list'> ['IntentResult']
<class 'list'> ['SmsAddressGenerator']
<class 'list'> ['EmailGenerator']
<class 'list'> ['WifiGenerator']
<class 'list'> ['Generator']
<class 'list'> ['StylesDefs']
<class 'list'> ['GeneratorException']
<class 'list'> ['StringConstants']
<class 'list'> ['UrlGenerator']
<class 'list'> ['TimeZoneList', 'TimeZoneInf

# 训练和 验证数据的物理生成步骤，生成真实可用的数据集
load_data->get_example
get_example:生成list型数据0<CODESPLIT>95536<CODESPLIT>org.aspectj/modules/weaver/src/org/aspectj/weaver/patterns/ITokenSource.java<CODESPLIT>pos_summ_descrip<CODESPLIT>code
load_data：将train_pos 和train_neg数据合并然后分成训练集和验证集
code_pre:主要目的是去除所有的注释 ，import， 替换回车换行符 ，过滤掉词长度小于3，最终返回数据格式是list

In [10]:
def code_pre(outstring):
    #主要目的是去除所有的注释 ，import， 替换回车换行符 ，过滤掉词长度小于3，最终返回数据格式是list
    m = re.compile(r'/\*.*?\*/', re.S)
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m = re.compile(r'//.*')
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m=re.compile(r'import.*')
    outtmp=re.sub(m,'',outstring)
    outstring = outtmp
    m = re.compile(r'package.*')
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m = re.compile(r'#.*')
    outtmp = re.sub(m, ' ', outstring)
    outstring = outtmp
    for char in ['\r\n', '\r', '\n']:
        outstring = outstring.replace(char, ' ')
    list_str=outstring.split()
    list_str=[t for t in list_str if len(t)>=3]
    # outstring = ' '.join(list_str)
    return list_str

In [11]:
def get_example(spamreader,examples,report_data,clean_path,codeDict,reportDict):#codeDict查询文件对应的class_method_names
    '''
        spamreader,读取的是该文件里的数据data/hdj/cross_project_trans/report/aspectj_80_training_pos.csv 格式：
            id,28974
            path,org.aspectj/modules/weaver/src/org/aspectj/weaver/patterns/ITokenSource.java
            label，0
        report_data,读取的是该文件 /data/hdj/cross_project_trans/report/aspectj_report_clean.csv，得到clean后的report信息
        clean_path,clean后的code的目录地址'/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/'
        codeDict,#单独提取code的className 和methodName信息，是dict格式
        reportDict,#单独提取的report的带词性的summary 和description的信息，dict格式
        
        return examples,返回生成的样本
    '''
    i = 0
    for row in spamreader:
        if i == 0:#为了跳过首行的标题
            i += 1
            continue
        # bug_report_name = "JDT_CNN/BugCorpus/BugCorpus/" + row[0].strip()
        # 根据bugid获得对应report
        bug_report_id = row[0].strip()
        # bug_description = getReport(bug_report_id, report_data)
        pos_summ_descrip=reportDict[bug_report_id]
        source_code_name = row[1].strip()
        #拼装成 /home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/org.aspectj/modules/weaver/src/org/aspectj/weaver/patterns/ITokenSource.java
        code_path = clean_path + source_code_name
        try:
            rf = open(code_path, 'r', encoding='utf-8', errors='ignore')
            data = rf.read()
        except:
            # print(data)
#             rf.close()
            print(code_path,'不存在*******')
            continue
        
        try:#如果code可以使用ast解析出来就使用className和methodName
            code=codeDict[source_code_name]+' '
        except:#否则使用原始的code文本数据
            # code=''
            code = ' '.join(code_pre(data))
        # code += ' '.join(code_pre(data))

        label = row[2].strip()
        tmp = []
        tmp.append(label)
        tmp.append(bug_report_id)  # TODO 记录report_id
        tmp.append(source_code_name)
        tmp.append(pos_summ_descrip)
        tmp.append(code)
        #数据格式是0<CODESPLIT>95536<CODESPLIT>org.aspectj/modules/weaver/src/org/aspectj/weaver/patterns/ITokenSource.java<CODESPLIT>pos_summ_descrip<CODESPLIT>code
        examples.append("<CODESPLIT>".join(tmp))
    return examples


In [12]:
def load_data( report_path, project_name, more_project_name, clean_path,tag,codeDict,reportDict):
    ''' 
        report_path, '/data/hdj/cross_project_trans/report/'
        project_name, aspectj_
        more_project_name, 'aspectj_80_'
        clean_path,'/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/'
        tag,'train'
        codeDict,dict格式，存放code的单独提取出来的信息
        reportDict，dict格式，单独提取的report的带词性的summary 和description的信息，
    '''
    # labels_list, report_ids_list, path_list,report_list,code_list = [], [], [],[],[]
    examples=[]
    #拼装成该目录 /data/hdj/cross_project_trans/report/aspectj_report_clean.csv，得到clean后的report信息
    report_data = pd.read_csv(report_path + project_name + 'report_clean.csv')
    #读取pos的数据
    #拼装成/data/hdj/cross_project_trans/report/aspectj_80_training_pos.csv
    pos_spamreader = csv.reader(open(report_path + more_project_name + "training_pos.csv", newline=''))
    examples=get_example(pos_spamreader,examples,report_data,clean_path,codeDict,reportDict)
    #读取neg的数据
    #拼装成/data/hdj/cross_project_trans/report/aspectj_80_training_neg.csv
    neg_spamreader = csv.reader(open(report_path + more_project_name + "training_neg.csv", newline=''))
    examples = get_example(neg_spamreader, examples, report_data, clean_path,codeDict,reportDict)
    
    #aspectj_train.txt
    #保存数据
    if tag=='train':
        train_set, val_set = train_test_split(examples, test_size=0.2, random_state=42)
        #拼装成 /data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_train.txt
        with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/'+project_name+tag+'.txt','w',encoding='utf-8') as f_out:
            for example in train_set:
                f_out.write(example+'\n')
        #/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_val.txt
        with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/'+project_name+'val'+'.txt','w',encoding='utf-8') as f_out:
            for example in val_set:
                f_out.write(example+'\n')
#     else:
#         #/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_test.txt 这里未必在使用，应该单独写好了test集的生成过程
#         with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/'+project_name+tag+'.txt','w',encoding='utf-8') as f_out:
#             for example in examples:
#                 f_out.write(example+'\n')

In [22]:

pathLessPosData = csv.reader(open('/data/hdj/cross_project_trans/report/aspectj_80_training_pos.csv', newline=''))

In [20]:
pathLessPosData

In [16]:
all_data=[]

In [23]:
# all_data.append(pathLessPosData)
for i,row in enumerate(pathLessPosData):
    print(row)
    if i==2:
        break

['id', 'path', 'label']
['28974', 'org.aspectj/modules/weaver/src/org/aspectj/weaver/AjcMemberMaker.java', '1']
['28919', 'org.aspectj/modules/weaver/src/org/aspectj/weaver/bcel/LazyMethodGen.java', '1']


In [18]:
all_data

In [13]:


# def append_data(path,all_data):
#     '''
#         path:加载数据的目录
#         all_data:用于返回所有的数据
#     '''
#     data = csv.reader(open(path, newline=''))
#     for i, row in enumerate(data):
#         if i==0:
#             continue
#         all_data.append(row)
#     return all_data
def getCode_Report_dict(project_name):
        code = open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/'+project_name+'code_class_method_names.pickle', 'rb')
        codeDict = pickle.load(code)
        #单独提取的report的带词性的summary 和description的信息，dict格式
        report = open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/'+project_name+'report_pos_summ_descr.pickle','rb')
        reportDict = pickle.load(report)
        return codeDict,reportDict
def load_data_new( report_path, project_name, train_project, code_clean_dict,codeDict,reportDict):
    ''' 
        report_path, '/data/hdj/cross_project_trans/report/'
        project_name, aspectj_
        train_project, ['swt_','zxing_']
        code_clean_dict,{'aspectj_':'/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/'}
        codeDict,dict格式，存放code的单独提取出来的信息
        reportDict，dict格式，单独提取的report的带词性的summary 和description的信息，
    '''
    # labels_list, report_ids_list, path_list,report_list,code_list = [], [], [],[],[]
    examples=[]
    #拼装成该目录 /data/hdj/cross_project_trans/report/aspectj_report_clean.csv，得到clean后的report信息
    report_data = pd.read_csv(report_path + project_name + 'report_clean.csv')
    #读取pos的数据
    #拼装成/data/hdj/cross_project_trans/report/aspectj_80_training_pos.csv
    for train_pro in train_project:
        codeDict,reportDict=getCode_Report_dict(train_pro)
       
        pathLessPos=report_path+train_pro+'20_training_pos.csv'
        all_data = csv.reader(open(pathLessPos, newline=''))
        examples=get_example(all_data,examples,report_data,code_clean_dict[train_pro],codeDict,reportDict)
       
        pathLessNeg=report_path+train_pro+'20_training_neg.csv'
        all_data = csv.reader(open(pathLessNeg, newline=''))
        examples=get_example(all_data,examples,report_data,code_clean_dict[train_pro],codeDict,reportDict)
        
        pathMorePos=report_path+train_pro+'80_training_neg.csv'
        all_data = csv.reader(open(pathMorePos, newline=''))
        examples=get_example(all_data,examples,report_data,code_clean_dict[train_pro],codeDict,reportDict)
        
        pathMoreNeg=report_path+train_pro+'80_training_neg.csv'
        all_data = csv.reader(open(pathMoreNeg, newline=''))
        examples=get_example(all_data,examples,report_data,code_clean_dict[train_pro],codeDict,reportDict)
        
        
    codeDict,reportDict=getCode_Report_dict(project_name)
    
    targetPos=report_path+project_name+'20_training_pos.csv'
    all_data = csv.reader(open(targetPos, newline=''))
    examples=get_example(all_data,examples,report_data,code_clean_dict[project_name],codeDict,reportDict)
    
    targetNeg=report_path+project_name+'20_training_neg.csv'
    all_data = csv.reader(open(targetNeg, newline=''))
    examples=get_example(all_data,examples,report_data,code_clean_dict[project_name],codeDict,reportDict)
    print(len(examples),'长度为')
    #aspectj_train.txt
    #保存数据
    train_set, val_set = train_test_split(examples, test_size=0.2, random_state=42)
    #拼装成 /data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_train.txt
    with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/'+project_name+'train'+'.txt','w',encoding='utf-8') as f_out:
        for example in train_set:
            f_out.write(example+'\n')
    #/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_val.txt
    with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/'+project_name+'val'+'.txt','w',encoding='utf-8') as f_out:
        for example in val_set:
            f_out.write(example+'\n')


In [15]:
if __name__=="__main__":
    #项目名
    project_names=['aspectj_','swt_','zxing_']# project_names=[]
    train_project=[['swt_','zxing_'],['aspectj_','zxing_'],['aspectj_','swt_']]
    #training_pos,neg 80 20 数据存放地址
    report_path = '/data/hdj/cross_project_trans/report/'
    #clean后代码存放位置
    code_clean_dict = {'aspectj_':'/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/',
                 'swt_':'/home/hdj/bug_localization/data/SWT/SWT-3.1/src/',
                 'zxing_':'/home/hdj/bug_localization/data/ZXing/ZXing-1.6/'}
    i = 0
   
    
    for project_name,train_pro in zip(project_names,train_project):
        #单独提取code的className 和methodName信息，是dict格式
       
        load_data_new( report_path,project_name,train_pro, code_clean_dict,codeDict=None,reportDict=None)
        i += 1
        # pickle.dump([train_report, train_code, train_labels, W],open("parameters.in", "wb"))
        # pickle.dump([train_report, train_code, train_labels, W],open("JDT_CNN/parameters.in", "wb"))
        print("train Finish processing!")

996 长度为
train Finish processing!
2344 长度为
train Finish processing!
2618 长度为
train Finish processing!


#  废弃的代码

In [ ]:
#废弃的代码
if __name__=="__main__":
    #项目名
    project_names=['aspectj_']# project_names=['aspectj_','swt_','zxing_']
    #training_pos,neg 80 20 数据存放地址
    report_path = '/data/hdj/cross_project_trans/report/'
    #clean后代码存放位置
    code_clean = ['/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/']#/home/hdj/bug_localization/data/SWT/clean/', '/home/hdj/bug_localization/data/ZXing/clean/']
    i = 0
    #单独提取code的className 和methodName信息，是dict格式
    code = open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/code_class_method_names.pickle', 'rb')
    codeDict = pickle.load(code)
    #单独提取的report的带词性的summary 和description的信息，dict格式
    report = open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/report_pos_summ_descr.pickle','rb')
    reportDict = pickle.load(report)
    
    for project_name in project_names:
       
        load_data( report_path,project_name, project_name + '80_', code_clean[i],'train',codeDict,reportDict)
        i += 1
        # pickle.dump([train_report, train_code, train_labels, W],open("parameters.in", "wb"))
        # pickle.dump([train_report, train_code, train_labels, W],open("JDT_CNN/parameters.in", "wb"))
        print("train Finish processing!")
    # i = 0
    # for project_name in project_names:
    #     load_data( report_path,project_name, project_name + '20_', code_clean[i],'test')
    #     i += 1
    #     # pickle.dump([train_report, train_code, train_labels, W],open("parameters.in", "wb"))
    #     # pickle.dump([train_report, train_code, train_labels, W],open("JDT_CNN/parameters.in", "wb"))
    #     print("val Finish processing!")
    # i = 0
    # for project_name in project_names:
    #     load_data( report_path,project_name, project_name + '80_', code_clean[i],'train')
    #     i += 1
    #     # pickle.dump([train_report, train_code, train_labels, W],open("parameters.in", "wb"))
    #     # pickle.dump([train_report, train_code, train_labels, W],open("JDT_CNN/parameters.in", "wb"))
    #     print("train Finish processing!")

# test数据集的物理生成步骤

In [16]:
def load_data_test( report_path, project_name, index, clean_path,test_path,codeDict,reportDict,out_path):
    # labels_list, report_ids_list, path_list,report_list,code_list = [], [], [],[],[]
    examples=[]
    report_data = pd.read_csv(report_path + project_name + 'report_clean.csv')
    pos_spamreader = csv.reader(open(test_path, newline=''))
    examples=get_example(pos_spamreader,examples,report_data,clean_path,codeDict,reportDict)

    #aspectj_train.txt

    with open(out_path+project_name+str(index)+'.txt','w',encoding='utf-8') as f_out:
        for example in examples:
            f_out.write(example+'\n')

In [17]:
if __name__=="__main__":
    # project_names=['aspectj_','swt_','zxing_']
    report_path = '/data/hdj/cross_project_trans/report/'
    
    project_names=['aspectj_','swt_','zxing_']
    # project_names=['swt_']
    # project_names=['zxing_']

    code_clean = ['/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/','/home/hdj/bug_localization/data/SWT/SWT-3.1/src/','/home/hdj/bug_localization/data/ZXing/ZXing-1.6/']
    # code_clean = ['/home/hdj/bug_localization/data/SWT/SWT-3.1/src/']
    # code_clean = ['/home/hdj/bug_localization/data/ZXing/ZXing-1.6/']
                 #'/home/hdj/bug_localization/data/SWT/clean/' ,'/home/hdj/bug_localization/data/SWT/clean/', '/home/hdj/bug_localization/data/ZXing/clean/']

    pre_test_paths=['/data/hdj/data/CodeBERT/data/codesearch/test/aspectj_test/','/data/hdj/data/CodeBERT/data/codesearch/test/swt_test',
                    '/data/hdj/data/CodeBERT/data/codesearch/test/zxing_test']
    # pre_test_path='/data/hdj/data/CodeBERT/data/codesearch/test/swt_test'
    # pre_test_path='/data/hdj/data/CodeBERT/data/codesearch/test/zxing_test'

    i = 0
    projectDicts=['aspectj_','swt_','zxing_']
    # projectDict='swt_'
    # projectDict='zxing_'

    totals = [58,20,4]
#     totals=[228,78,16]
    # total = 78
    # total = 16

    out_paths=['/data/hdj/data/CodeBERT/data/codesearch/test/aspectj_test/aspectj/','/data/hdj/data/CodeBERT/data/codesearch/test/swt_test/swt/',
              '/data/hdj/data/CodeBERT/data/codesearch/test/zxing_test/zxing/']
    # out_path = '/data/hdj/data/CodeBERT/data/codesearch/test/swt_test/swt/'
    # out_path='/data/hdj/data/CodeBERT/data/codesearch/test/zxing_test/zxing/'
    mergePaths=['> /data/hdj/data/CodeBERT/data/codesearch/test/aspectj_test/aspectj/','> /data/hdj/data/CodeBERT/data/codesearch/test/swt_test/swt/',
                '> /data/hdj/data/CodeBERT/data/codesearch/test/zxing_test/zxing/']
    for project_name in project_names:
        code = open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/' + projectDicts[i] + 'code_class_method_names.pickle',
                    'rb')
        codeDict = pickle.load(code)
        report = open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/parser_values/' + projectDicts[i] + 'report_pos_summ_descr.pickle',
                      'rb')
        reportDict = pickle.load(report)
        for index in range(totals[i]):
            test_path = os.path.join(pre_test_paths[i], 'test' + str(index) + '.csv')
            # print(test_path)
            load_data_test( report_path,project_name, index, code_clean[i],test_path,codeDict,reportDict,out_paths[i])

        # cat b1.sql b2.sql b3.sql > b_all.sql
        string='cat '
        for k in range(totals[i]):
            string+=out_paths[i]+project_names[i]+str(k)+'.txt '
        string += mergePaths[i]+'test_all_0417.txt'
        # print(string)
        os.system(string)
        i += 1
        print("test Finish processing!")
#         if i==2:
#             break

test Finish processing!
test Finish processing!
test Finish processing!


# 需要进行oversample

In [54]:
import numpy as np
file_path_train='/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_train_oversample.txt'
file_path_val='/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_val_oversample.txt'
# with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_val.txt','r',encoding='utf-8') as f_in:

with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_train.txt','r',encoding='utf-8') as f_in:
    with open(file_path_train, 'w', encoding='utf-8') as f:

        examples=[]
        for line in f_in:
            line=line.strip()
            if line[0]=='1':
                # print(line)
                examples+=[line for _ in range(30)]
            else:
                examples.append(line)
        idxs = np.arange(len(examples))
        examples = np.array(examples, dtype=np.object)

        np.random.seed(0)  # set random seed so that random things are reproducible
        np.random.shuffle(idxs)
        examples = examples[idxs]

        # print(examples)
        f.writelines('\n'.join(examples))

with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_val.txt','r',encoding='utf-8') as f_in:
    with open(file_path_val, 'w', encoding='utf-8') as f:

        examples=[]
        for line in f_in:
            line=line.strip()
            if line[0]=='1':
                # print(line)
                examples+=[line for _ in range(30)]
            else:
                examples.append(line)
        idxs = np.arange(len(examples))
        examples = np.array(examples, dtype=np.object)

        np.random.seed(0)  # set random seed so that random things are reproducible
        np.random.shuffle(idxs)
        examples = examples[idxs]

        # print(examples)
        f.writelines('\n'.join(examples))

with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_val_oversample.txt', 'r', encoding='utf-8') as f_in:
    count = 0
    total = 0
    for line in f_in:
        line = line.strip()
        if (line[0] == '1'):
            count += 1
        total += 1
    print('val :', count / total,count,total)
with open('/data/hdj/data/CodeBERT/data/codesearch/train_valid/aspectj_train_oversample.txt', 'r', encoding='utf-8') as f_in:
    count = 0
    total = 0
    for line in f_in:
        line = line.strip()
        if (line[0] == '1'):
            count += 1
        total += 1
    print('train :', count / total,count,total)

val : 0.2703826955074875 1950 7212
train : 0.26730708990289237 7680 28731


In [2]:
(20250/5)*4,(20250/5)

(16200.0, 4050.0)

In [5]:
with open('/data/hdj/data/CodeBERT/codesearch/results/java/0415_aspectj_summary_code_256_result_all.txt','r',encoding='utf-8') as f_in:
    for i,line in enumerate(f_in):
#         print(line)
        contents=line.split('<CODESPLIT>')
        print('label :',contents[0],' bug_id :',contents[1],' label_0_score :',contents[5],' label_1_socre :',contents[6])
        if(i==1000):
            break

label : 1  bug_id : 251326  label_0_score : 3.025240898132324  label_1_socre : -3.22164249420166

label : 1  bug_id : 251326  label_0_score : 2.9882595539093018  label_1_socre : -3.160094976425171

label : 1  bug_id : 251326  label_0_score : 2.627492904663086  label_1_socre : -2.801480770111084

label : 0  bug_id : 251326  label_0_score : 2.9914920330047607  label_1_socre : -3.248964548110962

label : 0  bug_id : 251326  label_0_score : 2.9394142627716064  label_1_socre : -3.134219169616699

label : 0  bug_id : 251326  label_0_score : 2.835158586502075  label_1_socre : -3.0136027336120605

label : 0  bug_id : 251326  label_0_score : 3.024888277053833  label_1_socre : -3.25238037109375

label : 0  bug_id : 251326  label_0_score : 2.999284505844116  label_1_socre : -3.2705581188201904

label : 0  bug_id : 251326  label_0_score : 2.8946707248687744  label_1_socre : -3.047170400619507

label : 0  bug_id : 251326  label_0_score : 2.679354190826416  label_1_socre : -2.860163927078247

label 

label : 0  bug_id : 251326  label_0_score : 2.7153825759887695  label_1_socre : -2.954868793487549

label : 0  bug_id : 251326  label_0_score : 2.825319290161133  label_1_socre : -3.0646510124206543

label : 0  bug_id : 251326  label_0_score : 2.9511332511901855  label_1_socre : -3.179189682006836

label : 0  bug_id : 251326  label_0_score : 2.6900036334991455  label_1_socre : -2.8583037853240967

label : 0  bug_id : 251326  label_0_score : 2.9584856033325195  label_1_socre : -3.216693878173828

label : 0  bug_id : 251326  label_0_score : 2.991062879562378  label_1_socre : -3.221112012863159

label : 0  bug_id : 251326  label_0_score : 2.997718572616577  label_1_socre : -3.2540135383605957

label : 0  bug_id : 251326  label_0_score : 2.8641202449798584  label_1_socre : -3.013812303543091

label : 0  bug_id : 251326  label_0_score : 3.0020296573638916  label_1_socre : -3.2529540061950684

label : 0  bug_id : 251326  label_0_score : 2.976003885269165  label_1_socre : -3.167968511581421



label : 0  bug_id : 251326  label_0_score : 2.9295482635498047  label_1_socre : -3.1667094230651855

label : 0  bug_id : 251326  label_0_score : 2.163478136062622  label_1_socre : -2.3219900131225586

label : 0  bug_id : 251326  label_0_score : 2.510075092315674  label_1_socre : -2.7159361839294434

label : 0  bug_id : 251326  label_0_score : 2.9371414184570312  label_1_socre : -3.181535243988037

label : 0  bug_id : 251326  label_0_score : 2.910696029663086  label_1_socre : -3.132030963897705

label : 0  bug_id : 251326  label_0_score : 2.9544625282287598  label_1_socre : -3.164879560470581

label : 0  bug_id : 251326  label_0_score : 3.0007996559143066  label_1_socre : -3.249694347381592

label : 0  bug_id : 251326  label_0_score : 2.5143587589263916  label_1_socre : -2.6672170162200928

label : 0  bug_id : 251326  label_0_score : 3.0059099197387695  label_1_socre : -3.264892578125

label : 0  bug_id : 251326  label_0_score : 2.9287056922912598  label_1_socre : -3.1803009510040283

l

In [2]:
aspectj_more_train=pd.read_pickle('/home/hdj/cross_project_trans/report/aspectj_more_train_data.pkl')

In [4]:
l1=['1','2']
l2=['3','4']
l3=['5','6']
for a,b,c in zip(l1,l2,l3):
    print(a,b,c)

1 3 5
2 4 6


In [15]:
def code_merge(raw_code_list):
    code_list=[]
    for raw_code in raw_code_list:
        try:
            code_list.append(raw_code['text'])
        except:
            print('出错了')
            print(raw_code)
    code_text=' '.join(code_list)
    return code_text

def data_tensor_generate(data,project,tokenizer,maxLength=512,ttype='train'):
    '''
        输入是dataframe 包含bug_id	cnn_report_idx	code_idx	label	path	raw_report	raw_code
    '''
    #将raw_code里dict形式合并生成code_text
    data['raw_code_text']=data['raw_code'].apply(code_merge)
    #生成完成
    
    
    if ttype=='train':
        from sklearn.model_selection import StratifiedShuffleSplit
        if False:
            split = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 42)
            X=np.array(data[['bug_id','path','raw_report','raw_code_text']].values)
            y=np.array(data['label'].values)
            ss = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 42)
            for train_index,test_index in split.split(X,y):#['bug_id','path','raw_report','raw_code_text'] ['label']
                train_set = X[train_index]
                train_label=y[train_index]
                val_set = X[test_index]
                val_label=y[test_index]
            with open('/home/hdj/cross_project_trans/report/'+project+'train/train.txt','w',encoding='utf-8') as f_out:
                all_labels=list(train_label)
                all_bug_id=list(train_set[:,0])
                all_path=list(train_set[:,1])
                raw_report=list(train_set[:,2])
                raw_code=list(train_set[:,3])
            #             all_data=[]
                print(len(all_labels),len(all_bug_id),len(all_path),len(raw_report),len(raw_code))
                for label,bug_id,path,report,code in zip(all_labels,all_bug_id,all_path,raw_report,raw_code):
                    string=str(label)+'<CODESPLIT>'+str(bug_id)+'<CODESPLIT>'+path+'<CODESPLIT>'+report+'<CODESPLIT>'+code
    #                 all_data.append(string)
                    f_out.write(string+'\n')
            with open('/home/hdj/cross_project_trans/report/'+project+'train/val.txt','w',encoding='utf-8') as f_out:
                all_labels=list(val_label)
                all_bug_id=list(val_set[:,0])
                all_path=list(val_set[:,1])
                raw_report=list(val_set[:,2])
                raw_code=list(val_set[:,3])
    #             all_data=[]
                for label,bug_id,path,report,code in zip(all_labels,all_bug_id,all_path,raw_report,raw_code):
                    string=str(label)+'<CODESPLIT>'+str(bug_id)+'<CODESPLIT>'+path+'<CODESPLIT>'+report+'<CODESPLIT>'+code
    #                 all_data.append(string)
                    f_out.write(string+'\n')
        else:
            with open('/home/hdj/cross_project_trans/report/'+project+'train/train.txt','w',encoding='utf-8') as f_out:
                all_labels=list(data['label'])
                all_bug_id=list(data['bug_id'])
                all_path=list(data['path'])
                raw_report=list(data['raw_report'])
                raw_code=list(data['raw_code_text'])
    #             all_data=[]
                for label,bug_id,path,report,code in zip(all_labels,all_bug_id,all_path,raw_report,raw_code):
                    string=str(label)+'<CODESPLIT>'+str(bug_id)+'<CODESPLIT>'+path+'<CODESPLIT>'+report+'<CODESPLIT>'+code
    #                 all_data.append(string)
                    f_out.write(string+'\n')
    else:
        with open('/home/hdj/cross_project_trans/report/'+project+'test_middle/all_test.txt','w',encoding='utf-8') as f_out:
            all_labels=list(data['label'])
            all_bug_id=list(data['bug_id'])
            all_path=list(data['path'])
            raw_report=list(data['raw_report'])
            raw_code=list(data['raw_code_text'])
#             all_data=[]
            for label,bug_id,path,report,code in zip(all_labels,all_bug_id,all_path,raw_report,raw_code):
                string=str(label)+'<CODESPLIT>'+str(bug_id)+'<CODESPLIT>'+path+'<CODESPLIT>'+report+'<CODESPLIT>'+code
#                 all_data.append(string)
                f_out.write(string+'\n')
#             for string in all_data:
                

In [13]:
#aspectj test数据生成
all_data=pd.DataFrame()
for i in range(58):
    data=pd.read_pickle('/home/hdj/cross_project_trans/report/aspectj_test_middle/test'+str(i)+'.pkl')
#     print(data.shape)
    all_data=pd.concat((all_data,data),axis=0)
data_tensor_generate(all_data,project='aspectj_',tokenizer=None,maxLength=512,ttype='test')

In [16]:
#aspectj train数据生成
data=pd.read_pickle('/home/hdj/cross_project_trans/report/aspectj_more_train_data.pkl')
data_tensor_generate(data,project='aspectj_',tokenizer=None,maxLength=512,ttype='train')

In [7]:
# src='/data/hdj/SourceFile/data/BuGL/Java/Projects/dubbo-master'#2089
# src='/data/hdj/SourceFile/data/BuGL/Java/Projects/jib-master'#443
src='/data/hdj/SourceFile/data/BuGL/Java/Projects/RxJava-3.x'#1851
src_addresses = glob.glob(str(src) + '/**/*.java', recursive=True) #文件不为空
print(len(src_addresses))

1851


In [64]:
from sklearn.model_selection import StratifiedShuffleSplit
data=pd.DataFrame()
data['bug_id']=['1','2','3','4','5','6']
data['raw_report']=['a','b','c','d','e','6']
data['path']=['q','w','e','r','t','6']
data['raw_code_text']=['11','22','33','44','55','66']
data['label']=[1,1,1,0,0,0]
print(data)
X=np.array(data[['bug_id','path','raw_report','raw_code_text']].values)
y=np.array(data['label'].values)
ss = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 42)
for train_index,test_index in split.split(X,y):#['bug_id','path','raw_report','raw_code_text'] ['label']
    train_set = X[train_index]
    train_label=y[train_index]
    val_set = X[test_index]
    val_label=y[test_index]
    print(train_index)
print(train_set)
print(train_label)
print(val_set)
print(val_label)

  bug_id raw_report path raw_code_text  label
0      1          a    q            11      1
1      2          b    w            22      1
2      3          c    e            33      1
3      4          d    r            44      0
4      5          e    t            55      0
5      6          6    6            66      0
[4 2 3 1]
[['5' 't' 'e' '55']
 ['3' 'e' 'c' '33']
 ['4' 'r' 'd' '44']
 ['2' 'w' 'b' '22']]
[0 1 0 1]
[['1' 'q' 'a' '11']
 ['6' '6' '6' '66']]
[1 0]


In [95]:
bug_id=train_set[:,0]
path=train_set[:,1]
for id,pa in zip(bug_id,path):
    s=id+'<code>'+pa
    print(id,pa,s)

5 t 5<code>t
3 e 3<code>e
4 r 4<code>r
2 w 2<code>w


In [57]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1,2]])
Y = np.array([0, 0, 0, 1, 1])
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=23) 

for train_index, test_index in ss.split(X, Y):
    print(train_index, test_index)
    print(X[train_index,:])

[0 1 3] [2 4]
[[1 2]
 [3 4]
 [3 4]]


In [45]:
data[['bug_id','path','raw_report','raw_code_text']].values
data['label'].values
X=np.array([[1,2],[3,4],[5,6],[3,3]])
Y=np.array([1,1,0,1])

[3 1] [4 2 0]
[2 3] [0 1 4]
[3 0] [4 1 2]
[4 0] [3 2 1]
[4 0] [2 3 1]


In [41]:
np.unique(Y)

array([0, 1])

In [43]:
train_index,test_index = ss.split(X,Y)#['bug_id','path','raw_report','raw_code_text'] ['label']
#     train_set = data[train_index,:]
#     val_set = data[test_index,:]
print(train_index)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [3]:
aspectj_more_train.head()

,bug_id,cnn_report_idx,code_idx,label,raw_report,raw_code,path
0,28974,"[0, 0, 0, 0, 9, 51, 43, 56, 26, 36, 55, 59, 54...","[[67, 4278, 66, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,compiler error introducing field aspect fails ...,"[{'y': 1, 'text': 'org aspectj weaver', 'num_w...",org.aspectj/modules/weaver/src/org/aspectj/wea...
1,28919,"[0, 0, 0, 0, 4455, 171, 154, 163, 145, 162, 13...","[[67, 4278, 66, 4458, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,waever tries weave methods find exception bug ...,"[{'y': 1, 'text': 'org aspectj weaver bcel', '...",org.aspectj/modules/weaver/src/org/aspectj/wea...
2,29186,"[0, 0, 0, 0, 34, 4797, 301, 4440, 316, 4789, 8...","[[67, 4278, 66, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,ajc emacssym chokes pointcut includes intertyp...,"[{'y': 1, 'text': 'org aspectj weaver', 'num_w...",org.aspectj/modules/weaver/src/org/aspectj/wea...
3,29186,"[0, 0, 0, 0, 34, 4797, 301, 4440, 316, 4789, 8...","[[67, 4278, 66, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,ajc emacssym chokes pointcut includes intertyp...,"[{'y': 1, 'text': 'org aspectj weaver', 'num_w...",org.aspectj/modules/weaver/src/org/aspectj/wea...
4,29186,"[0, 0, 0, 0, 34, 4797, 301, 4440, 316, 4789, 8...","[[67, 4278, 66, 4458, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,ajc emacssym chokes pointcut includes intertyp...,"[{'y': 1, 'text': 'org aspectj weaver bcel', '...",org.aspectj/modules/weaver/src/org/aspectj/wea...
